In [1]:
import os
import matplotlib.pyplot as plt

plt.style.use("default")

os.chdir("/home/wpkim/snuh")
print(os.getcwd())

/home/wpkim/snuh


In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from functools import reduce
from IPython.core.display import display, HTML

from scipy import stats

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

import glob
import shap
import joblib
import datetime

from tqdm import tqdm

import xgboost as xgb
import lightgbm as lgb
import multiprocessing as mp

import imblearn as imb
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

/home/wpkim/anaconda3/envs/py37_snuh/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class CoreUtils():
    def __init__(self):
        pass

    def check_label_ratio(self, df, column):
        neg, pos = np.bincount(df[column])
        total = neg + pos

        print("{}\n    Total: {}\n    Nagatives: {}\n    Positives: {} ({:.2f}% of dataframe)\n".format(
            column, total, neg, pos, (pos / total * 100)
        ))

    def check_valid_model(self, model, X_test, y_test, t_sen, t_spe, verbose=False):
        y_pred = model.predict_proba(X_test)[:, 1]
        y_pred = (y_pred >= 0.5).astype(int)

        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        sen = tp / (tp + fn)
        spe = tn / (tn + fp)

        if verbose:
            print("Sensitivity: {:.3f}, Specificity: {:.3f}".format(sen, spe))

        if (sen >= t_sen) & (spe >= t_spe):
            return True
        else:
            return False

    def compute_model_performance(self, model, X_test, y_test, youden=False):
        y_pred = model.predict_proba(X_test)[:, 1]

        fpr, tpr, thresholds = roc_curve(y_test, y_pred)
        auroc = auc(fpr, tpr)

        if youden:
            J = tpr - fpr
            ix = np.argmax(J)
            best_threshold = thresholds[ix]
            y_pred = (y_pred >= best_threshold).astype(int)
        else:
            y_pred = (y_pred >= 0.5).astype(int)

        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

        acc = (tp + tn) / (tp + fn + fp + tn)
        sen = tp / (tp + fn)
        spe = tn / (tn + fp)
        pre = tp / (tp + fp)
        f1 = (2 * pre * sen) / (pre + sen)
        npv = tn / (tn + fn)
        odd_ratio = (tp * tn) / (fp * fn)

        if youden:
            return pd.DataFrame([best_threshold, auroc, acc, pre, sen, spe, f1, npv, odd_ratio]).T
        else:
            return pd.DataFrame([auroc, acc, pre, sen, spe, f1, npv, odd_ratio]).T

    def compute_model_tprs(self, model, X_train, y_train, X_test, y_test):
        mean_fpr = np.linspace(0, 1, 100)

        train_tprs, test_tprs = [], []

        y_train_pred = model.predict_proba(X_train)[:, 1]
        y_test_pred = model.predict_proba(X_test)[:, 1]

        train_fpr, train_tpr, _ = roc_curve(y_train, y_train_pred)
        test_fpr, test_tpr, _ = roc_curve(y_test, y_test_pred)

        train_auroc = auc(train_fpr, train_tpr)
        test_auroc = auc(test_fpr, test_tpr)

        train_tprs.append(np.interp(mean_fpr, train_fpr, train_tpr))
        train_tprs[-1][0] = 0.0
        train_tprs = np.mean(train_tprs, axis=0)

        test_tprs.append(np.interp(mean_fpr, test_fpr, test_tpr))
        test_tprs[-1][0] = 0.0
        test_tprs = np.mean(test_tprs, axis=0)

        return pd.DataFrame([train_tprs, test_tprs, [train_auroc], [test_auroc]]).T

    def compute_shap_values(self, model, X_test, mode):
        feature_names = X_test.columns

        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(X_test)

        if mode == "rf" or mode == "lgb":
            model_shap = pd.DataFrame(
                list(
                    zip(
                        feature_names, np.abs(pd.DataFrame(shap_values[1], columns=feature_names).values).mean(0)
                    )
                ),
                columns=["columns", "shap_values"]
            )
        elif mode == "xgb":
            model_shap = pd.DataFrame(
                list(
                    zip(
                        feature_names, np.abs(pd.DataFrame(shap_values, columns=feature_names).values).mean(0)
                    )
                ),
                columns=["columns", "shap_values"]
            )

        return model_shap.sort_values(by=["shap_values"], ascending=False)

In [12]:
class SNUHStroke():
    def __init__(self):
        self.cores = CoreUtils()

        self.PATH_DATASET = None
        self.PATH_SAVE = None

        self.c_pre, c_post = [], []
        self.c_hours = []

        self.df = None
        self.df_ttest_tp = None
        self.df_ttest_mean = None
        self.df_ttest_std = None
        self.df_train = None
        self.df_test = None
        self.X_train, self.y_train = None, None
        self.X_test, self.y_test = None, None

    def set_path(self, path_dataset, path_save):
        self.PATH_DATASET = path_dataset
        self.PATH_SAVE = path_save

    def load_dataset(self):
        try:
            df = pd.read_csv(self.PATH_DATASET, sep=",")
        except:
            print("Error to open file")
            raise
        print(np.unique(df["caseid"]).shape)

        df["sum_rbc_my"] = df["sum_rbc_my"].fillna(0)
        df["sum_tf"] = df["sum_ffp"] + df["sum_plt"] + df["sum_cryo"] + df["sum_rbc_my"]
        df["sum_io"] = df["sum_fluid"] - (df["sum_uo"] + df["sum_ebl"])

        self.df = df.copy()

    def drop_exclusion(self):
        df = self.df.copy()

        ## ART
        query = "exclusion_art_not_exist == 1"
        df = df.drop(index=df.query(query).index)
        print(df.shape, df.query("is_stroke == 1").shape)

        self.cores.check_label_ratio(df, "is_stroke")

        ## DONOR
        query = "exclusion_cardiovascular_surgery_deceased_donor == 1"
        df = df.drop(index=df.query(query).index)
        print(df.shape, df.query("is_stroke == 1").shape)

        ## exclusion is_stroke 
        exclusion_keys = [
            5760, 9660, 11987, 11996, 12887, 13469, 15255, 15257, 16607, 17914, 19017,
            19613, 20023, 28712, 29092, 30743, 30872, 33686, 33751, 36408, 36409, 38631, 38680
        ]
        query = "caseid == @exclusion_keys"
        df = df.drop(index=df.query(query).index)
        print(df.shape, df.query("is_stroke == 1").shape)

        ## exclusion age
        query = "age < 18"
        df = df.drop(index=df.query(query).index)
        print(df.shape, df.query("is_stroke == 1").shape)

        ## exclusion weight
        query = "weight < 30 | weight >= 140"
        df = df.drop(index=df.query(query).index)
        print(df.shape, df.query("is_stroke == 1").shape)

        ## exclusion height
        query = "height < 135 | height >= 200"
        df = df.drop(index=df.query(query).index)
        print(df.shape, df.query("is_stroke == 1").shape)

        ## exclusion op duration
        query = "opdur_cal_sm2 <= 20"
        df = df.drop(index=df.query(query).index)
        print(df.shape, df.query("is_stroke == 1").shape)

        self.df = df.copy()

    def set_features(self):
        df = self.df.copy()


        self.c_pre = [
            "age", "sex",
            "height", "weight", "bmi",
            "preop_asa", "preop_emop",
            "preop_htn", "preop_dm", "preop_cva", "preop_asthma", "preop_copd",
            "preop_liverds", "preop_kidneyds", "preop_tb", "preop_hb_my", "preop_plt_sm",
            "preop_cr_sm", "preop_na", "preop_alb", "preop_bun", "preop_k_sm",
            "preop_pt", "preop_ptt_sm", "preop_glucose_sm", "preop_gpt_sm", "preop_got",
            "preop_egfr"
        ]
        self.c_post = [
            "opdur_cal_sm2", "andur_cal_my",
            "sum_uo", "sum_ebl", "sum_crystalloid", "sum_colloid", "sum_fluid", "sum_rbc_my",
            "sum_ffp", "sum_plt", "sum_cryo", "sum_tf", "sum_io",

            "SBP_below100", "SBP_below95", "SBP_below90", "SBP_below85", "SBP_below80",
            "SBP_below75", "SBP_below70", "SBP_below65", "SBP_below60", "SBP_below55",
            "SBP_below50", "SBP_mean", "SBP_std",

            "DBP_below70", "DBP_below65", "DBP_below60", "DBP_below55", "DBP_below50",
            "DBP_below45", "DBP_below40", "DBP_below35", "DBP_below30", "DBP_below25",
            "DBP_below20", "DBP_mean", "DBP_std",

            "SPO2_below95",
            "SPO2_mean", "SPO2_std",

            "HR_above100", "HR_above105", "HR_above110", "HR_above115", "HR_above120",
            "HR_above125", "HR_above130", "HR_above135", "HR_above140", "HR_above145",
            "HR_above150", "HR_below60", "HR_below55", "HR_below50", "HR_below45",
            "HR_below40", "HR_below35", "HR_mean", "HR_std",
        ]

        df = df[self.c_pre + self.c_post + ["opyear", "is_stroke"]]

        self.df = df.copy()

    def fill_features(self, mode, drop=True):
        df = self.df.copy()

        df["sex"] = pd.get_dummies(df["sex"])["M"]
        if not drop:
            df["preop_asa"] = df["preop_asa"].fillna(1)
        df["preop_emop"] = pd.get_dummies(df["preop_emop"].fillna("N"))["Y"]

        query = "preop_hb_my == '18.818.8'"
        df.loc[df.query(query).index, "preop_hb_my"] = 18.8
        df["preop_hb_my"] = df["preop_hb_my"].astype(float)

        query = "preop_na == '.'"
        df.loc[df.query(query).index, "preop_na"] = 0.0
        df["preop_na"] = df["preop_na"].astype(float)

        if drop:
            df = df.dropna()
        else:
            df = df.fillna(0.0)

        for column in df.columns:
            df[column] = df[column].astype(float)

        columns = [
            "SBP_below100", "SBP_below95", "SBP_below90", "SBP_below85", "SBP_below80",
            "SBP_below75", "SBP_below70", "SBP_below65", "SBP_below60", "SBP_below55",
            "SBP_below50", 

            "DBP_below70", "DBP_below65", "DBP_below60", "DBP_below55", "DBP_below50",
            "DBP_below45", "DBP_below40", "DBP_below35", "DBP_below30", "DBP_below25",
            "DBP_below20", 

            "SPO2_below95", 

            "HR_above100", "HR_above105", "HR_above110", "HR_above115", "HR_above120",
            "HR_above125", "HR_above130", "HR_above135", "HR_above140", "HR_above145",
            "HR_above150",
            "HR_below60", "HR_below55", "HR_below50", "HR_below45",
            "HR_below40", "HR_below35", 
        ]
        for column in columns:
            
            df[column] = (df[column] * 2) / 60

        if mode == "pp":
            df = df[self.c_pre + self.c_post + ["opyear", "is_stroke"]]
            print(df.shape)
        elif mode == "p":
            df = df[self.c_pre + ["opyear", "is_stroke"]]
            print(df.shape)
        else:
            print("??")
            raise

        self.df = df.copy()

    def compute_ttest(self, full=False):
        df = self.df.copy()

        if not full:
            query = "is_stroke == 0"
            df_control = df.query(query).drop(columns=["is_stroke"]).reset_index(drop=True)
            query = "is_stroke == 1"
            df_target = df.query(query).drop(columns=["is_stroke"]).reset_index(drop=True)

            li = []
            for column in df_control.columns:
                ret = stats.ttest_ind(df_control[column], df_target[column], equal_var=True)
                li.append([column, "{:.5f}".format(ret[0]), "{:.3f}".format(ret[1])])

            self.df_ttest_tp = pd.DataFrame(li, columns=["feature_name", "t_value", "p_value"])

            self.df_ttest_mean = pd.concat(
                [pd.DataFrame(np.mean(df_control), columns=["control"]),
                 pd.DataFrame(np.mean(df_target), columns=["is_stroke"])],
                axis=1
            )

            self.df_ttest_std = pd.concat(
                [pd.DataFrame(np.std(df_control), columns=["control"]),
                 pd.DataFrame(np.std(df_target), columns=["is_stroke"])],
                axis=1
            )
        else:
            self.df_ttest_mean = pd.DataFrame(np.mean(df))
            self.df_ttest_std = pd.DataFrame(np.std(df))

    def train_test_split(self):
        df = self.df.copy()

        query = "opyear < 19"
        self.df_train = df.query(query).reset_index(drop=True)
        query = "opyear == 19"
        self.df_test = df.query(query).reset_index(drop=True)
        print("Data shape: {} {}".format(self.df_train.shape, self.df_test.shape))

        self.X_train = self.df_train.drop(columns=["opyear", "is_stroke"])
        self.y_train = self.df_train["is_stroke"]
        self.X_test = self.df_test.drop(columns=["opyear", "is_stroke"])
        self.y_test = self.df_test["is_stroke"]
        print(self.X_train.shape, self.y_train.shape, self.X_test.shape, self.y_test.shape)

In [13]:
import warnings
warnings.filterwarnings("ignore")  
np.seterr(divide = 'ignore')  

path_dataset = os.path.join(os.getcwd(), "data", "220321_SNUH_vitaldb_n42306_full.csv")

stroke_pp = SNUHStroke()
stroke_pp.set_path(path_dataset, None)
stroke_pp.load_dataset()
stroke_pp.drop_exclusion()
stroke_pp.set_features()
stroke_pp.fill_features(mode="pp", drop=True)
stroke_pp.compute_ttest(full=False)
stroke_pp.train_test_split()


(42306,)
(18797, 135) (134, 135)
is_stroke
    Total: 18797
    Nagatives: 18663
    Positives: 134 (0.71% of dataframe)

(18783, 135) (134, 135)
(18769, 135) (128, 135)
(18631, 135) (127, 135)
(18615, 135) (127, 135)
(18601, 135) (127, 135)
(18504, 135) (127, 135)
(15752, 91)
Data shape: (11557, 91) (4195, 91)
(11557, 89) (11557,) (4195, 89) (4195,)


In [14]:
stroke_pp.df.head()

,age,sex,height,weight,bmi,preop_asa,preop_emop,preop_htn,preop_dm,preop_cva,...,HR_below60,HR_below55,HR_below50,HR_below45,HR_below40,HR_below35,HR_mean,HR_std,opyear,is_stroke
0,77.0,1.0,160.2,65.5,25.5,2.0,0.0,1.0,1.0,0.0,...,19.200000,0.000000,0.000000,0.000000,0.000000,0.0,77.192667,14.608640,16.0,0.0
9,72.0,1.0,162.0,66.7,25.4,3.0,0.0,1.0,1.0,0.0,...,0.200000,0.033333,0.000000,0.000000,0.000000,0.0,75.705077,4.341007,17.0,0.0
11,46.0,0.0,169.2,88.2,30.8,2.0,0.0,0.0,0.0,0.0,...,8.000000,4.333333,2.733333,2.233333,1.833333,1.5,80.599449,10.643429,16.0,0.0
16,85.0,1.0,164.2,53.7,19.9,2.0,0.0,0.0,0.0,0.0,...,13.733333,2.166667,0.100000,0.000000,0.000000,0.0,72.586622,8.166579,17.0,0.0
18,74.0,1.0,171.3,65.4,22.3,2.0,0.0,0.0,1.0,0.0,...,13.833333,4.200000,1.200000,0.700000,0.166667,0.0,74.848265,8.051744,17.0,0.0


In [15]:
print(stroke_pp.X_train.shape, stroke_pp.X_test.shape)

(11557, 89) (4195, 89)


In [16]:
import scipy.stats as stats

df_train = stroke_pp.X_train.copy()
df_train["trainable"] = 1
df_train["is_stroke"] = stroke_pp.y_train
df_test = stroke_pp.X_test.copy()
df_test["trainable"] = 0
df_test["is_stroke"] = stroke_pp.y_test

df_dataset = pd.concat([df_train, df_test], axis=0)

cat_columns = [
    "is_stroke", "sex", "preop_asa", "preop_emop", "preop_htn", "preop_dm", "preop_cva",
    "preop_asthma", "preop_copd", "preop_liverds", "preop_kidneyds", "preop_tb"
]
li = []
for column in cat_columns:
    data = pd.crosstab(df_dataset[column], df_dataset.trainable)
    chi = stats.chi2_contingency(observed=data)
    li.append([column, chi[0], chi[1], chi[2]])  ## stats, p-value, dof (degrees of freedom), expeted
pd.DataFrame(li)

,0,1,2,3
0,is_stroke,4.241625,3.944440e-02,1
1,sex,16.825080,4.098800e-05,1
2,preop_asa,81.270010,4.550787e-16,5
3,preop_emop,11.723445,6.171760e-04,1
4,preop_htn,1.611807,2.042380e-01,1
5,preop_dm,0.043918,8.340056e-01,1
6,preop_cva,1.252195,2.631336e-01,1
7,preop_asthma,0.070955,7.899517e-01,1
8,preop_copd,0.628552,4.278868e-01,1
9,preop_liverds,30.217473,3.862143e-08,1


In [23]:
stroke_pp.df_ttest_mean

,control,is_stroke
age,58.507000,58.917431
sex,0.493448,0.467890
height,162.223359,162.094495
weight,63.451007,62.702752
bmi,24.037052,23.738532
...,...,...
HR_below40,0.282405,0.381651
HR_below35,0.064148,0.058716
HR_mean,71.054365,71.638298
HR_std,9.337982,9.633512


In [22]:
stroke_pp.df_ttest_tp[["feature_name", "p_value"]]

,feature_name,p_value
0,age,0.772
1,sex,0.595
2,height,0.877
3,weight,0.514
4,bmi,0.398
...,...,...
85,HR_below40,0.628
86,HR_below35,0.907
87,HR_mean,0.618
88,HR_std,0.394


In [390]:
from scipy.stats import mannwhitneyu

query = "trainable == 1"
df_trainable = df_dataset.query(query)
query = "trainable == 0"
df_testable = df_dataset.query(query)

c_continuous = [
    "age",  "height", "weight", "bmi",
    "preop_hb_my", "preop_plt_sm",
    "preop_cr_sm", "preop_na", "preop_alb", "preop_bun", "preop_k_sm",
    "preop_pt", "preop_ptt_sm", "preop_glucose_sm", "preop_gpt_sm", "preop_got",
    "preop_egfr",

    "opdur_cal_sm2", "andur_cal_my",
    "sum_uo", "sum_ebl", "sum_crystalloid", "sum_colloid", "sum_fluid", "sum_rbc_my",
    "sum_ffp", "sum_plt", "sum_cryo", "sum_tf", "sum_io",

    "SBP_below100", "SBP_below95", "SBP_below90", "SBP_below85", "SBP_below80",
    "SBP_below75", "SBP_below70", "SBP_below65", "SBP_below60", "SBP_below55",
    "SBP_below50", "SBP_mean", "SBP_std",
    "DBP_below70", "DBP_below65", "DBP_below60", "DBP_below55", "DBP_below50",
    "DBP_below45", "DBP_below40", "DBP_below35", "DBP_below30", "DBP_below25",
    "DBP_below20", "DBP_mean", "DBP_std",
    "SPO2_below95", 
    "SPO2_mean", "SPO2_std",
    "HR_above100", "HR_above105", "HR_above110", "HR_above115", "HR_above120",
    "HR_above125", "HR_above130", "HR_above135", "HR_above140", "HR_above145",
    "HR_above150", "HR_below60", "HR_below55", "HR_below50", "HR_below45",
    "HR_below40", "HR_below35", "HR_mean", "HR_std",
]

li = []
df_control = stroke_pp.df.query("is_stroke == 0")
df_stroke = stroke_pp.df.query("is_stroke == 1")
for column in c_continuous:
    mann = mannwhitneyu(df_trainable[column], df_testable[column])

    li.append([column, mann[0], mann[1]])

pd.DataFrame(li, columns=["feature_name", "mann_statistics", "mann_p_value"])

,feature_name,mann_statistics,mann_p_value
0,age,24145977.5,7.069334e-01
1,height,24847326.0,1.620828e-02
2,weight,23949275.5,2.478446e-01
3,bmi,23511013.5,3.816660e-03
4,preop_hb_my,24241907.5,9.965221e-01
...,...,...,...
73,HR_below45,23572526.0,1.427867e-03
74,HR_below40,23500199.5,1.090158e-05
75,HR_below35,23591105.0,2.623113e-06
76,HR_mean,24972206.5,3.741454e-03


In [391]:
pd.concat([df_trainable.mean(), df_trainable.std(), df_testable.mean(), df_testable.std()], axis=1)

,0,1,2,3
age,58.467336,14.790802,58.626937,14.710310
sex,0.503158,0.500012,0.466031,0.498904
height,162.324600,8.671555,161.941097,8.658057
weight,63.346811,11.858415,63.718617,12.141394
bmi,23.967301,3.637135,24.221454,3.763371
...,...,...,...,...
HR_below35,0.050330,0.384408,0.102074,0.691011
HR_mean,71.251812,12.272677,70.525584,11.937226
HR_std,9.248408,3.529657,9.592434,3.790662
trainable,1.000000,0.000000,0.000000,0.000000


In [323]:
df = stroke_pp.df.reset_index(drop=True)

columns = [
    "SBP_below100", "SBP_below95", "SBP_below90", "SBP_below85", "SBP_below80",
    "SBP_below75", "SBP_below70", "SBP_below65", "SBP_below60", "SBP_below55",
    "SBP_below50", 
    "DBP_below70", "DBP_below65", "DBP_below60", "DBP_below55", "DBP_below50",
    "DBP_below45", "DBP_below40", "DBP_below35", "DBP_below30", "DBP_below25",
    "DBP_below20", 
    "HR_above100", "HR_above105", "HR_above110", "HR_above115", "HR_above120",
    "HR_above125", "HR_above130", "HR_above135", "HR_above140", "HR_above145",
    "HR_above150", "HR_below60", "HR_below55", "HR_below50", "HR_below45",
    "HR_below40", "HR_below35", 
]
for column in tqdm(columns):
    for p in range(df.shape[0]):
        df.loc[p, column] = (df.loc[p, column] / (df.loc[p, "opdur_cal_sm2"] * 30)) * 100.0

li = []
df_control = df.query("is_stroke == 0")
df_stroke = df.query("is_stroke == 1")

pd.concat([df_control.mean(), df_control.std(), df_stroke.mean(), df_stroke.std()], axis=1)

100%|██████████| 40/40 [01:09<00:00,  1.74s/it]


,0,1,2,3
age,58.507000,14.751273,58.917431,17.211715
sex,0.493448,0.499973,0.467890,0.501273
height,162.223359,8.663506,162.094495,9.512972
weight,63.451007,11.923424,62.702752,13.558006
bmi,24.037052,3.670339,23.738532,4.014624
...,...,...,...,...
HR_below35,0.044518,0.407983,0.044305,0.214875
HR_mean,71.054365,12.171509,71.638298,14.425643
HR_std,9.337982,3.599710,9.633512,4.195900
opyear,17.927124,0.845913,18.275229,0.621700


In [179]:
df = df_trainable
female, male = df["sex"].value_counts()[0], df["sex"].value_counts()[1]
print("{} {:.2f}   {} {:.2f}".format(female, female / df.shape[0] * 100, male, male / df.shape[0] * 100))

df = df_testable
female, male = df["sex"].value_counts()[0], df["sex"].value_counts()[1]
print("{} {:.2f}   {} {:.2f}".format(female, female / df.shape[0] * 100, male, male / df.shape[0] * 100))



5742 49.68   5815 50.32
2240 53.40   1955 46.60


In [150]:
column = "preop_asa"
print(column)

li = []
df = df_trainable
res = df[column].value_counts().reset_index()
res["percent"] = res[column] / df.shape[0] * 100
li.append(res.sort_values(by="index"))
pd.concat(li)



preop_asa


,index,preop_asa,percent
1,1.0,2411,20.861815
0,2.0,7163,61.979753
2,3.0,1849,15.998962
3,4.0,125,1.081596
4,5.0,6,0.051917
5,6.0,3,0.025958


In [151]:
li = []
df = df_testable
res = df[column].value_counts().reset_index()
res["percent"] = res[column] / df.shape[0] * 100
li.append(res.sort_values(by="index"))
pd.concat(li)

,index,preop_asa,percent
1,1.0,1150,27.413588
0,2.0,2327,55.470799
2,3.0,664,15.828367
3,4.0,49,1.168057
4,5.0,4,0.095352
5,6.0,1,0.023838


In [5]:
class CoreUtils():
    def __init__(self):
        pass


    def check_label_ratio(self, df, column):
        neg, pos = np.bincount(df[column])
        total = neg + pos

        print("{}\n    Total: {}\n    Nagatives: {}\n    Positives: {} ({:.2f}% of dataframe)\n".format(
            column, total, neg, pos, (pos / total * 100)
        ))


    def check_valid_model(self, model, X_test, y_test, t_sen, t_spe, verbose=False):
        y_pred = model.predict_proba(X_test)[:, 1]
        y_pred = (y_pred >= 0.5).astype(int)

        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        sen = tp / (tp + fn)
        spe = tn / (tn + fp)

        if verbose:
            print("Sensitivity: {:.3f}, Specificity: {:.3f}".format(sen, spe))

        if (sen >= t_sen) & (spe >= t_spe):
            return True
        else:
            return False


    def compute_model_performance(self, model, X_test, y_test, youden=False):
        y_pred = model.predict_proba(X_test)[:, 1]

        fpr, tpr, thresholds = roc_curve(y_test, y_pred)
        auroc = auc(fpr, tpr)

        if youden:
            J = tpr - fpr
            ix = np.argmax(J)
            best_threshold = thresholds[ix]

            y_pred = (y_pred >= best_threshold).astype(int)
        else:
            y_pred = (y_pred >= 0.5).astype(int)

        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

        acc = (tp + tn) / (tp + fn + fp + tn)
        sen = tp / (tp + fn)
        spe = tn / (tn + fp)
        pre = tp / (tp + fp)
        f1 = (2 * pre * sen) / (pre + sen)

        npv = tn / (tn + fn)
        odd_ratio = (tp * tn) / (fp * fn)

        if youden:
            return pd.DataFrame([best_threshold, auroc, acc, pre, sen, spe, f1, npv, odd_ratio]).T
        else:
            return pd.DataFrame([auroc, acc, pre, sen, spe, f1, npv, odd_ratio]).T


    def compute_model_tprs(self, model, X_test, y_test):
        mean_fpr = np.linspace(0, 1, 100)

        train_tprs, test_tprs = [], []

        y_test_pred = model.predict_proba(X_test)[:, 1]

        test_fpr, test_tpr, _ = roc_curve(y_test, y_test_pred)

        test_auroc = auc(test_fpr, test_tpr)

        test_tprs.append(np.interp(mean_fpr, test_fpr, test_tpr))
        test_tprs[-1][0] = 0.0
        test_tprs = np.mean(test_tprs, axis=0)

        return pd.DataFrame([test_tprs, [test_auroc]]).T


    def compute_shap_values(self, model, X_test, mode):
        feature_names = X_test.columns

        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(X_test)

        if mode == "rf" or mode == "lgb":
            model_shap = pd.DataFrame(
                list(
                    zip(
                        feature_names, np.abs(pd.DataFrame(shap_values[1], columns=feature_names).values).mean(0)
                    )
                ),
                columns=["columns", "shap_values"]
            )
        elif mode == "xgb":
            model_shap = pd.DataFrame(
                list(
                    zip(
                        feature_names, np.abs(pd.DataFrame(shap_values, columns=feature_names).values).mean(0)
                    )
                ),
                columns=["columns", "shap_values"]
            )

        return model_shap.sort_values(by=["shap_values"], ascending=False)

    def compute_youden_index(self, model, X_test, y_test):
        y_pred = model.predict_proba(X_test)[:, 1]

        fpr, tpr, thresholds = roc_curve(y_test, y_pred)
        auroc = auc(fpr, tpr)

        J = tpr - fpr
        ix = np.argmax(J)
        best_threshold = thresholds[ix]

        y_pred = (y_pred >= best_threshold).astype(int)
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

        sen = tp / (tp + fn)
        spe = tn / (tn + fp)

        ppv = tp / (tp + fp)
        npv = tn / (tn + tn)
        print("ppv={:.3f} npv={:.3f}".format(ppv, npv))

        odd_ratio = (tp * tn) / (fp * fn)
        print("odd_ratio={:.3f}".format(odd_ratio))

        mean_fpr = np.linspace(0, 1, 100)

        tprs = []
        tprs.append(np.interp(mean_fpr, fpr, tpr))
        tprs[-1][0] = 0.0
        tprs = np.mean(tprs, axis=0)

        return tprs, fpr, tpr, auroc, best_threshold, ix, sen, spe

In [6]:
class BRMHStroke():
    def __init__(self, kfold=True, model_save=False, debug=False, shap=False):
        self.cores = CoreUtils()

        self.PATH_DATASET = None
        self.PATH_SAVE = None

        self.WITH_KFOLD = kfold
        self.WITH_SAVE = model_save
        self.WITH_MODEL_DEBUG = debug
        self.WITH_SHAP = shap

        self.c_pre, c_post = [], []
        self.c_hours = []

        self.df = None

        self.df_ttest_tp = None
        self.df_ttest_mean = None
        self.df_ttest_std = None

        self.df_train = None
        self.df_test = None
        self.X_train, self.y_train = None, None
        self.X_test, self.y_test = None, None

        self.rf_summary, self.rf_tprs, self.rf_summary_youden, self.rf_shap = [], [], [], []
        self.xgb_summary, self.xgb_tprs, self.xgb_summary_youden, self.xgb_shap = [], [], [], []
        self.lgb_summary, self.lgb_tprs, self.lgb_summary_youden, self.lgb_shap = [], [], [], []


    def set_path(self, path_dataset, path_save):
        self.PATH_DATASET = path_dataset
        self.PATH_SAVE = path_save


    def load_dataset(self):
        try:
            df = pd.read_csv(self.PATH_DATASET, sep=",")
        except:
            print("Error to open file")
            raise
        print(df.shape)
        print(df.columns)

        df["sum_rbc_my"] = df["sum_rbc_my"].fillna(0)
        df["sum_tf"] = df["sum_ffp"] + df["sum_plt"] + df["sum_cryo"] + df["sum_rbc_my"]
        df["sum_io"] = df["sum_fluid"] - (df["sum_uo"] + df["sum_ebl"])

        self.df = df.copy()


    def drop_exclusion(self):
        df = self.df.copy()

        ## exclusion age
        query = "age < 18"
        df = df.drop(index=df.query(query).index)
        print("ex a >> ", df.shape)

        ## exclusion weight
        query = "weight < 30 | weight >= 140"
        df = df.drop(index=df.query(query).index)
        print("ex w >> ", df.shape)

        ## exclusion height
        query = "height < 135 | height >= 200"
        df = df.drop(index=df.query(query).index)
        print("ex h >> ", df.shape)

        ## exclusion op duration
        query = "opdur_cal_sm <= 20"
        df = df.drop(index=df.query(query).index)
        print("ex o >> ", df.shape)

        self.df = df.copy()


    def set_features(self):
        df = self.df.copy()
        df = df.rename(columns={"opdur_cal_sm": "opdur_cal_sm2"})  ## bugfix

        self.c_basic = [
            "no", "ptno", "serialno", "opname"
        ]
        self.c_pre = [
            "age", "sex",
            "height", "weight", "bmi",
            "preop_asa", "preop_emop",
            "preop_htn", "preop_dm", "preop_cva", "preop_asthma", "preop_copd",
            "preop_liverds", "preop_kidneyds", "preop_tb", "preop_hb_my", "preop_plt_sm",
            "preop_cr_sm", "preop_na", "preop_alb", "preop_bun", "preop_k_sm",
            "preop_pt", "preop_ptt_sm", "preop_glucose_sm", "preop_gpt_sm", "preop_got",
            "preop_egfr"
        ]
        self.c_post = [
            "opdur_cal_sm2", "andur_cal_my",
            "sum_uo", "sum_ebl", "sum_crystalloid", "sum_colloid", "sum_fluid", "sum_rbc_my",
            "sum_ffp", "sum_plt", "sum_cryo", "sum_tf", "sum_io",

            "SBP_below100", "SBP_below95", "SBP_below90", "SBP_below85", "SBP_below80",
            "SBP_below75", "SBP_below70", "SBP_below65", "SBP_below60", "SBP_below55",
            "SBP_below50", "SBP_mean", "SBP_std",

            "DBP_below70", "DBP_below65", "DBP_below60", "DBP_below55", "DBP_below50",
            "DBP_below45", "DBP_below40", "DBP_below35", "DBP_below30", "DBP_below25",
            "DBP_below20", "DBP_mean", "DBP_std",

            "SPO2_below95", 
            "SPO2_mean", "SPO2_std",

            "HR_above100", "HR_above105", "HR_above110", "HR_above115", "HR_above120",
            "HR_above125", "HR_above130", "HR_above135", "HR_above140", "HR_above145",
            "HR_above150", "HR_below60", "HR_below55", "HR_below50", "HR_below45",
            "HR_below40", "HR_below35", "HR_mean", "HR_std",
        ]

        

        df = df[self.c_basic + self.c_pre + self.c_post + ["is_stroke"]]


        self.df = df.copy()


    def fill_features(self, mode, full=False):
        df = self.df.copy()


        print("fill features original >> ", df.shape)
        self.cores.check_label_ratio(df, "is_stroke")

        if not full:
           
            pass
        else:
            df = df.fillna(0.0)

        for column in df.columns:
            try:
                df[column] = df[column].astype(float)
            except:
                print(column)

        columns = [
            "SBP_below100", "SBP_below95", "SBP_below90", "SBP_below85", "SBP_below80",
            "SBP_below75", "SBP_below70", "SBP_below65", "SBP_below60", "SBP_below55",
            "SBP_below50", 

            "DBP_below70", "DBP_below65", "DBP_below60", "DBP_below55", "DBP_below50",
            "DBP_below45", "DBP_below40", "DBP_below35", "DBP_below30", "DBP_below25",
            "DBP_below20", 

            "SPO2_below95", 

            "HR_above100", "HR_above105", "HR_above110", "HR_above115", "HR_above120",
            "HR_above125", "HR_above130", "HR_above135", "HR_above140", "HR_above145",
            "HR_above150",
            "HR_below60", "HR_below55", "HR_below50", "HR_below45",
            "HR_below40", "HR_below35"
        ]
        for column in columns:

            df[column] = (df[column] * 2) / 60


        if mode == "pp":
            df = df[self.c_basic + self.c_pre + self.c_post + ["is_stroke"]]    ##########################
            print(df.shape)
        elif mode == "i":
            df = df[self.c_post + ["is_stroke"]]
            print(df.shape)
        elif mode == "p":
            df = df[self.c_pre + ["is_stroke"]]
            print(df.shape)
        else:
            print("??")
            raise

        df = df.dropna()
        print("drop >> ", df.shape)

        self.df = df.copy()


    def compute_ttest(self, full=False):
        df = self.df.copy()

        if not full:
            query = "is_stroke == 0"
            df_control = df.query(query).drop(columns=["is_stroke"]).reset_index(drop=True)
            query = "is_stroke == 1"
            df_target = df.query(query).drop(columns=["is_stroke"]).reset_index(drop=True)

            li = []
            for column in df_control.columns:
                ret = stats.ttest_ind(df_control[column], df_target[column], equal_var=True)
                li.append([column, "{:.5f}".format(ret[0]), "{:.5f}".format(ret[1])])

            self.df_ttest_tp = pd.DataFrame(li, columns=["feature_name", "t_value", "p_value"])

            self.df_ttest_mean = pd.concat(
                [pd.DataFrame(np.mean(df_control), columns=["control"]),
                 pd.DataFrame(np.mean(df_target), columns=["is_stroke"])],
                axis=1
            )

            self.df_ttest_std = pd.concat(
                [pd.DataFrame(np.std(df_control), columns=["control"]),
                 pd.DataFrame(np.std(df_target), columns=["is_stroke"])],
                axis=1
            )
        else:
            self.df_ttest_mean = pd.DataFrame(np.mean(df))
            self.df_ttest_std = pd.DataFrame(np.std(df))


    def train_test_split(self):
        df = self.df.copy()

        self.X_test = df.drop(columns=["is_stroke"])
        self.y_test = df["is_stroke"]

        print(self.X_test.shape, self.y_test.shape)


    def forward(self, n_folds, n_epochs, n_cores, t_sen, t_spe, p_rf, p_xgb, p_lgb):
        print(datetime.datetime.now())

        df_train, df_test = self.df_train.copy(), self.df_test.copy()

        p_rf, p_xgb, p_lgb = p_rf, p_xgb, p_lgb
        while (p_rf < n_epochs) | (p_xgb < n_epochs) | (p_lgb < n_epochs):
            print(p_rf, p_xgb, p_lgb)

            rf_model = RandomForestClassifier(
                n_estimators=100,  max_depth=9,  bootstrap=False,
                criterion="gini",  max_features=None,  max_samples=None,  n_jobs=n_cores,
            )
            xgb_model = xgb.XGBClassifier(
                objective="binary:logistic",  n_estimators=100,  max_depth=9,
                verbosity=0,  booster="gbtree",  use_label_encoder=False,  tree_method="gpu_hist",
                eval_metric="auc",  n_jobs=n_cores,
            )
            lgb_model = lgb.LGBMClassifier(
                objective="binary",  n_estimators=200,  max_depth=9,
                boosting_type="gbdt",  metric="auc",  n_jobs=n_cores,
            )

            rf_tprs, xgb_tprs, lgb_tprs = [], [], []
            rf_aucs, xgb_aucs, lgb_aucs = [], [], []

            if self.WITH_KFOLD:
                folds = StratifiedKFold(n_splits=n_folds)

                for train_idx, valid_idx in folds.split(self.X_train, self.y_train):
                    X_trainset, y_trainset = self.X_train.iloc[train_idx], self.y_train.iloc[train_idx]
                    X_validset, y_validset = self.X_train.iloc[valid_idx], self.y_train.iloc[valid_idx]

                    undersampler = RandomUnderSampler(
                        replacement=True, sampling_strategy="majority",
                        random_state=np.random.RandomState()
                    )
                    X_trainset, y_trainset = undersampler.fit_resample(X_trainset, y_trainset)

                    if p_rf < n_epochs:
                        rf_model.fit(X_trainset, y_trainset)
                    if p_xgb < n_epochs:
                        xgb_model.fit(X_trainset, y_trainset)
                    if p_lgb < n_epochs:
                        lgb_model.fit(X_trainset, y_trainset)

            ## Check performance and get summaries
            if p_rf < n_epochs:
                if self.cores.check_valid_model(rf_model, self.X_test, self.y_test, t_sen-0.05, t_spe-0.05, verbose=self.WITH_MODEL_DEBUG):
                    if self.WITH_SAVE:
                        joblib.dump(rf_model, "{}_rf_ep{:02d}.pkl".format(self.PATH_SAVE, p_rf))

                    self.rf_summary.append(self.cores.compute_model_performance(rf_model, self.X_test, self.y_test))
                    self.rf_tprs.append(self.cores.compute_model_tprs(rf_model, self.X_train, self.y_train, self.X_test, self.y_test))
                    self.rf_summary_youden.append(self.cores.compute_model_performance(rf_model, self.X_test, self.y_test, youden=True))

                    if self.WITH_SHAP:
                        self.rf_shap.append(self.cores.compute_shap_values(rf_model, self.X_test, "rf"))

                    p_rf = p_rf + 1

            if p_xgb < n_epochs:
                if self.cores.check_valid_model(xgb_model, self.X_test, self.y_test, t_sen, t_spe, verbose=self.WITH_MODEL_DEBUG):
                    if self.WITH_SAVE:
                        joblib.dump(xgb_model, "{}_xgb_ep{:02d}.pkl".format(self.PATH_SAVE, p_xgb))

                    self.xgb_summary.append(self.cores.compute_model_performance(xgb_model, self.X_test, self.y_test))
                    self.xgb_tprs.append(self.cores.compute_model_tprs(xgb_model, self.X_train, self.y_train, self.X_test, self.y_test))
                    self.xgb_summary_youden.append(self.cores.compute_model_performance(xgb_model, self.X_test, self.y_test, youden=True))

                    if self.WITH_SHAP:
                        self.xgb_shap.append(self.cores.compute_shap_values(xgb_model, self.X_test, "xgb"))

                    p_xgb = p_xgb + 1

            if p_lgb < n_epochs:
                if self.cores.check_valid_model(lgb_model, self.X_test, self.y_test, t_sen, t_spe, verbose=self.WITH_MODEL_DEBUG):
                    if self.WITH_SAVE:
                        joblib.dump(lgb_model, "{}_lgb_ep{:02d}.pkl".format(self.PATH_SAVE, p_lgb))

                    self.lgb_summary.append(self.cores.compute_model_performance(lgb_model, self.X_test, self.y_test))
                    self.lgb_tprs.append(self.cores.compute_model_tprs(lgb_model, self.X_train, self.y_train, self.X_test, self.y_test))
                    self.lgb_summary_youden.append(self.cores.compute_model_performance(lgb_model, self.X_test, self.y_test, youden=True))

                    if self.WITH_SHAP:
                        self.lgb_shap.append(self.cores.compute_shap_values(lgb_model, self.X_test, "lgb"))

                    p_lgb = p_lgb + 1

        print(datetime.datetime.now())


    def recheck(self, PATH_MODEL_SAVED):
        self.rf_summary, self.rf_tprs, self.rf_summary_youden, self.rf_shap = [], [], [], []
        self.xgb_summary, self.xgb_tprs, self.xgb_summary_youden, self.xgb_shap = [], [], [], []
        self.lgb_summary, self.lgb_tprs, self.lgb_summary_youden, self.lgb_shap = [], [], [], []

        li_rf, li_xgb, li_lgb = [], [], []
        for file in glob.glob(os.path.join(PATH_MODEL_SAVED)):
            if "rf_" in file:
                li_rf.append(file)
            elif "xgb_" in file:
                li_xgb.append(file)
            elif "lgb_" in file:
                li_lgb.append(file)
            else:
                pass
        li_rf.sort(), li_xgb.sort(), li_lgb.sort()

        for file in li_rf:
            print(file)
            model = joblib.load(file)

            self.rf_summary.append(self.cores.compute_model_performance(model, self.X_test, self.y_test))
            self.rf_tprs.append(self.cores.compute_model_tprs(model, self.X_test, self.y_test))
            self.rf_summary_youden.append(self.cores.compute_model_performance(model, self.X_test, self.y_test, youden=True))

            if self.WITH_SHAP:
                self.rf_shap.append(self.cores.compute_shap_values(model, self.X_test, "rf"))

        for file in li_xgb:
            print(file)
            model = joblib.load(file)

            self.xgb_summary.append(self.cores.compute_model_performance(model, self.X_test, self.y_test))
            self.xgb_tprs.append(self.cores.compute_model_tprs(model, self.X_test, self.y_test))
            self.xgb_summary_youden.append(self.cores.compute_model_performance(model, self.X_test, self.y_test, youden=True))

            if self.WITH_SHAP:
                self.xgb_shap.append(self.cores.compute_shap_values(model, self.X_test, "xgb"))

        for file in li_lgb:
            print(file)
            model = joblib.load(file)

            self.lgb_summary.append(self.cores.compute_model_performance(model, self.X_test, self.y_test))
            self.lgb_tprs.append(self.cores.compute_model_tprs(model, self.X_test, self.y_test))
            self.lgb_summary_youden.append(self.cores.compute_model_performance(model, self.X_test, self.y_test, youden=True))

            if self.WITH_SHAP:
                self.lgb_shap.append(self.cores.compute_shap_values(model, self.X_test, "lgb"))

    def plot_youden_index(self, rf_path, xgb_path, lgb_path, title):
        rf_model = joblib.load(rf_path)
        xgb_model = joblib.load(xgb_path)
        lgb_model = joblib.load(lgb_path)

        rf_tprs, rf_fpr, rf_tpr, rf_auroc, rf_thr, rf_ix, rf_sen, rf_spe = self.cores.compute_youden_index(rf_model, self.X_test, self.y_test)
        xgb_tprs, xgb_fpr, xgb_tpr, xgb_auroc, xgb_thr, xgb_ix, xgb_sen, xgb_spe = self.cores.compute_youden_index(xgb_model, self.X_test, self.y_test)
        lgb_tprs, lgb_fpr, lgb_tpr, lgb_auroc, lgb_thr, lgb_ix, lgb_sen, lgb_spe = self.cores.compute_youden_index(lgb_model, self.X_test, self.y_test)

        lw = 1

        FONT_SIZE = 10
        plt.rc("font", size=FONT_SIZE) # controls default text sizes
        plt.rc("axes", titlesize=FONT_SIZE) # fontsize of the axes title
        plt.rc("axes", labelsize=FONT_SIZE) # fontsize of the x and y labels
        plt.rc("xtick", labelsize=FONT_SIZE) # fontsize of the tick labels
        plt.rc("ytick", labelsize=FONT_SIZE) # fontsize of the tick labels
        plt.rc("legend", fontsize=FONT_SIZE) # legend fontsize
        plt.rc("figure", titlesize=FONT_SIZE) # fontsize of the figure title

        color_r, color_g, color_b = "crimson", "darkgreen", "darkblue"

        plt.figure(figsize=(7, 6))

        plt.plot(rf_fpr, rf_tpr, linestyle="-", lw=lw, alpha=1, color="k")
        plt.plot(xgb_fpr, xgb_tpr, linestyle="-", lw=lw, alpha=1, color="k")
        plt.plot(lgb_fpr, lgb_tpr, linestyle="-", lw=lw, alpha=1, color="k")

        plt.scatter(rf_fpr[rf_ix], rf_tpr[rf_ix], marker="+", s=100, lw=3, color=color_r)
        plt.scatter(xgb_fpr[xgb_ix], xgb_tpr[xgb_ix], marker="+", s=100, lw=3, color=color_g)
        plt.scatter(lgb_fpr[lgb_ix], lgb_tpr[lgb_ix], marker="+", s=100, lw=3, color=color_b)

        plt.plot([0, 1], [0, 1], linestyle="--", lw=lw, color="k")
        plt.xlim([-0.05, 1.05])
        plt.ylim([-0.05, 1.05])
        plt.xlabel("1-Specificity")
        plt.ylabel("Sensitivity")
        plt.title(title)

        red_patch = mpatches.Patch(
            color=color_r,
            label="RandomForest\nAUC={:.3f} Th={:.3f} Se={:.3f} Sp={:.3f}".format(rf_auroc, rf_thr, rf_sen, rf_spe)
        )
        green_patch = mpatches.Patch(
            color=color_g,
            label="XGBoost\nAUC={:.3f} Th={:.3f} Se={:.3f} Sp={:.3f}".format(xgb_auroc, xgb_thr, xgb_sen, xgb_spe)
        )
        blue_patch = mpatches.Patch(
            color=color_b,
            label="LightGBM\nAUC={:.3f} Th={:.3f} Se={:.3f} Sp={:.3f}".format(lgb_auroc, lgb_thr, lgb_sen, lgb_spe)
        )

        plt.legend(loc="lower right", handles=[red_patch, green_patch, blue_patch])
        plt.show()


    # AUC comparison adapted from
    # https://github.com/Netflix/vmaf/
    def compute_midrank(self, x):
        """Computes midranks.
        Args:
           x - a 1D numpy array
        Returns:
           array of midranks
        """
        J = np.argsort(x)
        Z = x[J]
        N = len(x)
        T = np.zeros(N, dtype=np.float)
        i = 0
        while i < N:
            j = i
            while j < N and Z[j] == Z[i]:
                j += 1
            T[i:j] = 0.5*(i + j - 1)
            i = j
        T2 = np.empty(N, dtype=np.float)
        # Note(kazeevn) +1 is due to Python using 0-based indexing
        # instead of 1-based in the AUC formula in the paper
        T2[J] = T + 1
        return T2


    def fastDeLong(self, predictions_sorted_transposed, label_1_count):
        """
        The fast version of DeLong's method for computing the covariance of
        unadjusted AUC.
        Args:
           predictions_sorted_transposed: a 2D numpy.array[n_classifiers, n_examples]
              sorted such as the examples with label "1" are first
        Returns:
           (AUC value, DeLong covariance)
        Reference:
         @article{sun2014fast,
           title={Fast Implementation of DeLong's Algorithm for
                  Comparing the Areas Under Correlated Receiver Operating Characteristic Curves},
           author={Xu Sun and Weichao Xu},
           journal={IEEE Signal Processing Letters},
           volume={21},
           number={11},
           pages={1389--1393},
           year={2014},
           publisher={IEEE}
         }
        """
        # Short variables are named as they are in the paper
        m = label_1_count
        n = predictions_sorted_transposed.shape[1] - m
        positive_examples = predictions_sorted_transposed[:, :m]
        negative_examples = predictions_sorted_transposed[:, m:]
        k = predictions_sorted_transposed.shape[0]

        tx = np.empty([k, m], dtype=np.float)
        ty = np.empty([k, n], dtype=np.float)
        tz = np.empty([k, m + n], dtype=np.float)
        for r in range(k):
            tx[r, :] = self.compute_midrank(positive_examples[r, :])
            ty[r, :] = self.compute_midrank(negative_examples[r, :])
            tz[r, :] = self.compute_midrank(predictions_sorted_transposed[r, :])
        aucs = tz[:, :m].sum(axis=1) / m / n - float(m + 1.0) / 2.0 / n
        v01 = (tz[:, :m] - tx[:, :]) / n
        v10 = 1.0 - (tz[:, m:] - ty[:, :]) / m
        sx = np.cov(v01)
        sy = np.cov(v10)
        delongcov = sx / m + sy / n
        return aucs, delongcov


    def calc_pvalue(self, aucs, sigma):
        """Computes log(10) of p-values.
        Args:
           aucs: 1D array of AUCs
           sigma: AUC DeLong covariances
        Returns:
           log10(pvalue)
        """
        l = np.array([[1, -1]])
        z = np.abs(np.diff(aucs)) / np.sqrt(np.dot(np.dot(l, sigma), l.T))
        p = 2 * (1 - scipy.stats.norm.cdf(z, loc=0, scale=1))
        return z, p

    def compute_ground_truth_statistics(self, ground_truth):
        assert np.array_equal(np.unique(ground_truth), [0, 1])
        order = (-ground_truth).argsort()
        label_1_count = int(ground_truth.sum())
        return order, label_1_count


    def delong_roc_variance(self, ground_truth, predictions):
        """
        Computes ROC AUC variance for a single set of predictions
        Args:
           ground_truth: np.array of 0 and 1
           predictions: np.array of floats of the probability of being class 1
        """
        order, label_1_count = self.compute_ground_truth_statistics(ground_truth)
        predictions_sorted_transposed = predictions[np.newaxis, order]
        aucs, delongcov = self.fastDeLong(predictions_sorted_transposed, label_1_count)
        assert len(aucs) == 1, "There is a bug in the code, please forward this to the developers"
        return aucs[0], delongcov


    def delong_roc_test(self, ground_truth, predictions_one, predictions_two):
        """
        Computes log(p-value) for hypothesis that two ROC AUCs are different
        Args:
           ground_truth: np.array of 0 and 1
           predictions_one: predictions of the first model,
              np.array of floats of the probability of being class 1
           predictions_two: predictions of the second model,
              np.array of floats of the probability of being class 1
        """
        order, label_1_count = self.compute_ground_truth_statistics(ground_truth)
        predictions_sorted_transposed = np.vstack((predictions_one, predictions_two))[:, order]
        aucs, delongcov = self.fastDeLong(predictions_sorted_transposed, label_1_count)
        return self.calc_pvalue(aucs, delongcov)

    def print_delong_performance(self, p_path, pp_path, X_test_p, y_test_p, X_test_pp, y_test_pp):
        p_best_model = joblib.load(p_path)
        pp_best_model = joblib.load(pp_path)

        p_pred = p_best_model.predict_proba(X_test_p)[:, 1]
        pp_pred = pp_best_model.predict_proba(X_test_pp)[:, 1]


        p_pp_z, p_pp_p = self.delong_roc_test(y_test_p, p_pred, pp_pred)
        print(p_pp_z, p_pp_p)

        self.p_pp_delong_z = p_pp_z[0][0]
        self.p_pp_delong_p = p_pp_p[0][0]

    def print_delong_performance_with_threshold(self, p_path, pp_path, X_test_p, y_test_p, X_test_pp, y_test_pp, p_youden, pp_youden):
        p_best_model = joblib.load(p_path)
        pp_best_model = joblib.load(pp_path)

        p_pred = (p_best_model.predict_proba(X_test_p)[:, 1] >= p_youden).astype(int)
        pp_pred = (pp_best_model.predict_proba(X_test_pp)[:, 1] >= pp_youden).astype(int)

        p_pp_z, p_pp_p = self.delong_roc_test(y_test_p, p_pred, pp_pred)
        print(p_pp_z, p_pp_p)

        self.p_pp_delong_z = p_pp_z[0][0]
        self.p_pp_delong_p = p_pp_p[0][0]

In [7]:
path_dataset = os.path.join(os.getcwd(), "data", "220530_BRMH_vitaldb_n568_inclusion_stroke_filled_cut.csv")
path_save = "model_0524_final/stroke_p_cv10"

stroke = BRMHStroke(kfold=True, model_save=False, debug=False, shap=False)
stroke.set_path(path_dataset, path_save)
stroke.load_dataset()
stroke.cores.check_label_ratio(stroke.df, "is_stroke")
stroke.drop_exclusion()
stroke.cores.check_label_ratio(stroke.df, "is_stroke")
stroke.set_features()
stroke.cores.check_label_ratio(stroke.df, "is_stroke")
stroke.fill_features(mode="pp", full=False)
stroke.cores.check_label_ratio(stroke.df, "is_stroke")

stroke.cores.check_label_ratio(stroke.df, "is_stroke")


(490, 103)
Index(['no', 'ptno', 'serialno', 'opname', 'is_stroke', 'age', 'sex', 'height',
       'weight', 'bmi',
       ...
       'HR_above145', 'HR_above150', 'HR_below60', 'HR_below55', 'HR_below50',
       'HR_below45', 'HR_below40', 'HR_below35', 'HR_mean', 'HR_std'],
      dtype='object', length=103)
is_stroke
    Total: 490
    Nagatives: 479
    Positives: 11 (2.24% of dataframe)

ex a >>  (490, 103)
ex w >>  (490, 103)
ex h >>  (490, 103)
ex o >>  (490, 103)
is_stroke
    Total: 490
    Nagatives: 479
    Positives: 11 (2.24% of dataframe)

is_stroke
    Total: 490
    Nagatives: 479
    Positives: 11 (2.24% of dataframe)

fill features original >>  (490, 94)
is_stroke
    Total: 490
    Nagatives: 479
    Positives: 11 (2.24% of dataframe)

serialno
opname
(490, 94)
drop >>  (449, 94)
is_stroke
    Total: 449
    Nagatives: 438
    Positives: 11 (2.45% of dataframe)

is_stroke
    Total: 449
    Nagatives: 438
    Positives: 11 (2.45% of dataframe)



In [395]:
import scipy.stats as stats

cat_columns = [
    "is_stroke", "sex", "preop_asa", "preop_emop", "preop_htn", "preop_dm", "preop_cva",
    "preop_asthma", "preop_copd", "preop_liverds", "preop_kidneyds", "preop_tb"
]
li = []
for column in cat_columns:
    data = pd.crosstab(stroke.df[column], stroke.df.is_stroke)
    chi = stats.chi2_contingency(observed=data)
    li.append([column, chi[0], chi[1], chi[2]])  ## stats, p-value, dof (degrees of freedom), expeted
pd.DataFrame(li)

,0,1,2,3
0,is_stroke,408.131571,9.350561e-91,1
1,sex,0.012599,9.106286e-01,1
2,preop_asa,3.744356,2.904252e-01,3
3,preop_emop,14.229356,1.618262e-04,1
4,preop_htn,3.091484,7.870300e-02,1
5,preop_dm,0.848877,3.568702e-01,1
6,preop_cva,13.716332,2.125977e-04,1
7,preop_asthma,0.000000,1.000000e+00,1
8,preop_copd,0.000000,1.000000e+00,1
9,preop_liverds,1.618845,2.032529e-01,1


In [396]:
from scipy.stats import mannwhitneyu

c_continuous = [
    "age",  "height", "weight", "bmi",
    "preop_hb_my", "preop_plt_sm",
    "preop_cr_sm", "preop_na", "preop_alb", "preop_bun", "preop_k_sm",
    "preop_pt", "preop_ptt_sm", "preop_glucose_sm", "preop_gpt_sm", "preop_got",
    "preop_egfr",

    "opdur_cal_sm2", "andur_cal_my",
    "sum_uo", "sum_ebl", "sum_crystalloid", "sum_colloid", "sum_fluid", "sum_rbc_my",
    "sum_ffp", "sum_plt", "sum_cryo", "sum_tf", "sum_io",

    "SBP_below100", "SBP_below95", "SBP_below90", "SBP_below85", "SBP_below80",
    "SBP_below75", "SBP_below70", "SBP_below65", "SBP_below60", "SBP_below55",
    "SBP_below50", "SBP_mean", "SBP_std",
    "DBP_below70", "DBP_below65", "DBP_below60", "DBP_below55", "DBP_below50",
    "DBP_below45", "DBP_below40", "DBP_below35", "DBP_below30", "DBP_below25",
    "DBP_below20", "DBP_mean", "DBP_std",
    "SPO2_below95", 
    "SPO2_mean", "SPO2_std",
    "HR_above100", "HR_above105", "HR_above110", "HR_above115", "HR_above120",
    "HR_above125", "HR_above130", "HR_above135", "HR_above140", "HR_above145",
    "HR_above150", "HR_below60", "HR_below55", "HR_below50", "HR_below45",
    "HR_below40", "HR_below35", "HR_mean", "HR_std",
]

li = []
df_control = stroke.df.query("is_stroke == 0")
df_stroke = stroke.df.query("is_stroke == 1")
for column in c_continuous:
   
    mann = mannwhitneyu(df_control[column], df_stroke[column])

    li.append([column, mann[0], mann[1]])

pd.DataFrame(li, columns=["feature_name", "mann_statistics", "mann_p_value"])

,feature_name,mann_statistics,mann_p_value
0,age,2374.5,0.936242
1,height,2051.0,0.400264
2,weight,2681.0,0.522988
3,bmi,2907.5,0.241355
4,preop_hb_my,2217.0,0.652272
...,...,...,...
73,HR_below45,3239.0,0.050995
74,HR_below40,3254.5,0.046813
75,HR_below35,3310.0,0.034128
76,HR_mean,1212.0,0.004879


In [397]:
pd.concat([df_control.mean(), df_control.std(), df_stroke.mean(), df_stroke.std()], axis=1)

,0,1,2,3
age,62.858904,14.207310,62.536364,16.990778
sex,0.518265,0.500238,0.454545,0.522233
height,160.862557,8.595372,163.727273,9.352229
weight,62.733562,11.637792,61.736364,15.558231
bmi,24.212236,3.912146,22.743276,3.588574
...,...,...,...,...
HR_below40,7.895053,6.029954,6.478788,9.197261
HR_below35,7.648250,5.807775,6.269697,9.127303
HR_mean,65.883462,13.736007,79.467102,14.819207
HR_std,23.672956,7.797121,16.332398,8.018057


In [376]:
df = stroke.df
query = "is_stroke == 0"
df = df.query(query)
female, male = df["sex"].value_counts()[0], df["sex"].value_counts()[1]
print("{} {:.2f}   {} {:.2f}".format(female, female / df.shape[0] * 100, male, male / df.shape[0] * 100))

df = stroke.df
query = "is_stroke == 1"
df = df.query(query)
female, male = df["sex"].value_counts()[0], df["sex"].value_counts()[1]
print("{} {:.2f}   {} {:.2f}".format(female, female / df.shape[0] * 100, male, male / df.shape[0] * 100))

211 48.17   227 51.83
6 54.55   5 45.45


In [17]:
column = "sex"
print(column)

li = []
queries = ["is_stroke == 0", "is_stroke == 1"]
for query in queries:
    df = stroke.df.query(query)

    res = df[column].value_counts().reset_index()
    res["percent"] = res[column] / df.shape[0] * 100
    li.append(res.sort_values(by="index"))
pd.concat(li)

sex


,index,sex,percent
1,0.0,211,48.173516
0,1.0,227,51.826484
0,0.0,6,54.545455
1,1.0,5,45.454545


In [64]:
column = "preop_gpt_sm"
print(column)

mean = stroke.df[column].mean()
std = stroke.df[column].std()
print("{:.2f} {:.2f}".format(mean, std))

preop_gpt_sm
28.17 32.85


In [47]:
column = "preop_"
print(column)

li = []

res = stroke.df[column].value_counts().reset_index()
res["percent"] = res[column] / stroke.df.shape[0] * 100
li.append(res.sort_values(by="index"))
pd.concat(li)

preop_na


,index,preop_na,percent
68,126.4,1,0.222717
72,128.8,1,0.222717
25,129.9,3,0.668151
77,130.0,1,0.222717
52,130.4,2,0.445434
...,...,...,...
9,144.0,6,1.336303
39,145.0,3,0.668151
50,146.6,2,0.445434
12,149.0,5,1.113586


In [28]:
stroke.df.columns

Index(['no', 'ptno', 'serialno', 'opname', 'age', 'sex', 'height', 'weight',
       'bmi', 'preop_asa', 'preop_emop', 'preop_htn', 'preop_dm', 'preop_cva',
       'preop_asthma', 'preop_copd', 'preop_liverds', 'preop_kidneyds',
       'preop_tb', 'preop_hb_my', 'preop_plt_sm', 'preop_cr_sm', 'preop_na',
       'preop_alb', 'preop_bun', 'preop_k_sm', 'preop_pt', 'preop_ptt_sm',
       'preop_glucose_sm', 'preop_gpt_sm', 'preop_got', 'preop_egfr',
       'opdur_cal_sm2', 'andur_cal_my', 'sum_uo', 'sum_ebl', 'sum_crystalloid',
       'sum_colloid', 'sum_fluid', 'sum_rbc_my', 'sum_ffp', 'sum_plt',
       'sum_cryo', 'sum_tf', 'sum_io', 'SBP_below100', 'SBP_below95',
       'SBP_below90', 'SBP_below85', 'SBP_below80', 'SBP_below75',
       'SBP_below70', 'SBP_below65', 'SBP_below60', 'SBP_below55',
       'SBP_below50', 'SBP_mean', 'SBP_std', 'DBP_below70', 'DBP_below65',
       'DBP_below60', 'DBP_below55', 'DBP_below50', 'DBP_below45',
       'DBP_below40', 'DBP_below35', 'DBP_below30', '

In [92]:
import scipy.stats as stats

df_snuh = stroke_pp.df.copy()
df_snuh["SNUH"] = 1
df_bmc = stroke.df.copy()
df_bmc["SNUH"] = 0

df_full = pd.concat([df_snuh, df_bmc], axis=0)

cat_columns = [
    "is_stroke", "sex", "preop_asa", "preop_emop", "preop_htn", "preop_dm", "preop_cva",
    "preop_asthma", "preop_copd", "preop_liverds", "preop_kidneyds", "preop_tb"
]
li = []
for column in cat_columns:
    data = pd.crosstab(df_full[column], df_full.SNUH)
    chi = stats.chi2_contingency(observed=data)
    li.append([column, chi[0], chi[1], chi[2]])  ## stats, p-value, dof (degrees of freedom), expeted
pd.DataFrame(li)

,0,1,2,3
0,is_stroke,16.036356,6.213777e-05,1
1,sex,0.867556,3.516329e-01,1
2,preop_asa,76.894108,3.742363e-15,5
3,preop_emop,0.006616,9.351703e-01,1
4,preop_htn,4.833645,2.790956e-02,1
5,preop_dm,19.855962,8.350188e-06,1
6,preop_cva,788.681171,1.559553e-173,1
7,preop_asthma,6.447785,1.110908e-02,1
8,preop_copd,2.685812,1.012456e-01,1
9,preop_liverds,32.622788,1.118951e-08,1


In [106]:

queries = ["is_stroke == 0", "is_stroke == 1"]
for query in queries:
    df = stroke.df.query(query)
    female, male = df["sex"].value_counts()[0], df["sex"].value_counts()[1]
    print("{} {:.2f}   {} {:.2f}".format(female, female / df.shape[0] * 100, male, male / df.shape[0] * 100))

211 48.17   227 51.83
6 54.55   5 45.45


In [99]:
df_snuh["is_stroke"].value_counts() / df_snuh.shape[0] * 100

0.0    99.308024
1.0     0.691976
Name: is_stroke, dtype: float64

In [117]:

column = "preop_tb"
print(column)

li = []
queries = ["is_stroke == 0", "is_stroke == 1"]
for query in queries:
    df = stroke.df.query(query)

    res = df[column].value_counts().reset_index()
    res["percent"] = res[column] / df.shape[0] * 100
    li.append(res.sort_values(by="index"))
pd.concat(li)

preop_tb


,index,preop_tb,percent
0,0.0,410,93.607306
1,1.0,28,6.392694
0,0.0,11,100.000000


In [120]:
import scipy.stats as stats

cat_columns = [
    "sex", "preop_asa", "preop_emop", "preop_htn", "preop_dm", "preop_cva",
    "preop_asthma", "preop_copd", "preop_liverds", "preop_kidneyds", "preop_tb"
]
li = []
for column in cat_columns:
    data = pd.crosstab(stroke.df[column], stroke.df.is_stroke)
    chi = stats.chi2_contingency(observed=data)
    li.append([column, chi[0], chi[1], chi[2]])  ## stats, p-value, dof (degrees of freedom), expeted
pd.DataFrame(li)

,0,1,2,3
0,sex,0.012599,0.910629,1
1,preop_asa,3.744356,0.290425,3
2,preop_emop,14.229356,0.000162,1
3,preop_htn,3.091484,0.078703,1
4,preop_dm,0.848877,0.356870,1
5,preop_cva,13.716332,0.000213,1
6,preop_asthma,0.000000,1.000000,1
7,preop_copd,0.000000,1.000000,1
8,preop_liverds,1.618845,0.203253,1
9,preop_kidneyds,0.307513,0.579211,1


In [121]:
from scipy.stats import mannwhitneyu

c_continuous = [
    "age",  "height", "weight", "bmi",
    "preop_hb_my", "preop_plt_sm",
    "preop_cr_sm", "preop_na", "preop_alb", "preop_bun", "preop_k_sm",
    "preop_pt", "preop_ptt_sm", "preop_glucose_sm", "preop_gpt_sm", "preop_got",
    "preop_egfr",

    "opdur_cal_sm2", "andur_cal_my",
    "sum_uo", "sum_ebl", "sum_crystalloid", "sum_colloid", "sum_fluid", "sum_rbc_my",
    "sum_ffp", "sum_plt", "sum_cryo", "sum_tf", "sum_io",

    "SBP_below100", "SBP_below95", "SBP_below90", "SBP_below85", "SBP_below80",
    "SBP_below75", "SBP_below70", "SBP_below65", "SBP_below60", "SBP_below55",
    "SBP_below50", "SBP_mean", "SBP_std",
    "DBP_below70", "DBP_below65", "DBP_below60", "DBP_below55", "DBP_below50",
    "DBP_below45", "DBP_below40", "DBP_below35", "DBP_below30", "DBP_below25",
    "DBP_below20", "DBP_mean", "DBP_std",
    "SPO2_below95", 
    "SPO2_mean", "SPO2_std",
    "HR_above100", "HR_above105", "HR_above110", "HR_above115", "HR_above120",
    "HR_above125", "HR_above130", "HR_above135", "HR_above140", "HR_above145",
    "HR_above150", "HR_below60", "HR_below55", "HR_below50", "HR_below45",
    "HR_below40", "HR_below35", "HR_mean", "HR_std",
]
query = "is_stroke == 0"
df_control = stroke.df.query(query)
query = "is_stroke == 1"
df_stroke = stroke.df.query(query)

li = []
for column in c_continuous:
    
    mann = mannwhitneyu(df_control[column], df_stroke[column])

    li.append([column, mann[0], mann[1]])

pd.DataFrame(li, columns=["feature_name", "mann_statistics", "mann_p_value"])

,feature_name,mann_statistics,mann_p_value
0,age,2374.5,0.936242
1,height,2051.0,0.400264
2,weight,2681.0,0.522988
3,bmi,2907.5,0.241355
4,preop_hb_my,2217.0,0.652272
...,...,...,...
73,HR_below45,3239.0,0.050995
74,HR_below40,3254.5,0.046813
75,HR_below35,3310.0,0.034128
76,HR_mean,1212.0,0.004879


In [6]:
from scipy.stats import norm
from statsmodels.formula.api import ols

c = [
    "age", "sex",
    "height", "weight", "bmi",
    "preop_asa", "preop_emop",
    "preop_htn", "preop_dm", "preop_cva", "preop_asthma", "preop_copd",
    "preop_liverds", "preop_kidneyds", "preop_tb", "preop_hb_my", "preop_plt_sm",
    "preop_cr_sm", "preop_na", "preop_alb", "preop_bun", "preop_k_sm",
    "preop_pt", "preop_ptt_sm", "preop_glucose_sm", "preop_gpt_sm", "preop_got",
    "preop_egfr",

    "opdur_cal_sm2", "andur_cal_my",
    "sum_uo", "sum_ebl", "sum_crystalloid", "sum_colloid", "sum_fluid", "sum_rbc_my",
    "sum_ffp", "sum_plt", "sum_cryo", "sum_tf", "sum_io",
    "SBP_below100", "SBP_below95", "SBP_below90", "SBP_below85", "SBP_below80",
    "SBP_below75", "SBP_below70", "SBP_below65", "SBP_below60", "SBP_below55",
    "SBP_below50", "SBP_mean", "SBP_std",
    "DBP_below70", "DBP_below65", "DBP_below60", "DBP_below55", "DBP_below50",
    "DBP_below45", "DBP_below40", "DBP_below35", "DBP_below30", "DBP_below25",
    "DBP_below20", "DBP_mean", "DBP_std",
    "SPO2_below95", "SPO2_below90", "SPO2_below85", "SPO2_below80", "SPO2_below75",
    "SPO2_below70", "SPO2_below65", "SPO2_below60", "SPO2_below55", "SPO2_below50",
    "SPO2_mean", "SPO2_std",
    "HR_above100", "HR_above105", "HR_above110", "HR_above115", "HR_above120",
    "HR_above125", "HR_above130", "HR_above135", "HR_above140", "HR_above145",
    "HR_above150", "HR_below60", "HR_below55", "HR_below50", "HR_below45",
    "HR_below40", "HR_below35", "HR_mean", "HR_std",
]

query = "is_stroke == 1"
fit = ols(formula="age ~ HR_std", data=stroke.df.query(query)[c]).fit()

sqrt_mse = np.sqrt(fit.mse_resid)
std_res = fit.resid / sqrt_mse

def empirical_dist(x, samples):
    count = 0
    for s in samples:
        if s <= x:
            count += 1
    return count / len(samples)

distance = []  
for r in std_res:
    diff = abs(empirical_dist(r, std_res) - norm.cdf(r))
    distance.append(diff)


print(1.94947 / np.sqrt(stroke.df.query(query).shape[0]))  
print(1.62762 / np.sqrt(stroke.df.query(query).shape[0]))  
print(1.35810 / np.sqrt(stroke.df.query(query).shape[0]))  
ks_stat = max(distance)
ks_stat  


0.1867253608250078
0.15589772183516504
0.13008238779588457


0.08473575964868885

In [14]:

from statsmodels.stats.diagnostic import kstest_normal

query = "is_stroke == 1"

li = []
for column in c:
    z, p = kstest_normal(stroke.df.query(query)[column], dist="norm")
    print(z, p)
print(np.max(pd.DataFrame(li).dropna()))  ## control = 0.001, stroke = 0.029

0.09100538853761786 0.033343003874257544
0.35680441837974314 0.0009999999999998899
0.08254446159964407 0.07580032671701631
0.06039210793628369 0.4555523182810275
0.07353595164588822 0.18475165735119173
0.2543344329715949 0.0009999999999998899
0.4898223753522974 0.0009999999999998899
0.3804760024718885 0.0009999999999998899
0.5203738049698934 0.0009999999999998899
0.5297000948080057 0.0009999999999998899
0.5289790808988767 0.0009999999999998899
0.5357757553025178 0.0009999999999998899
0.5405137635921748 0.0009999999999998899
0.5405137635921748 0.0009999999999998899
nan nan
0.09911729649408607 0.01367189761360533
0.10574447088864936 0.005723475413024716
0.23593869164790993 0.0009999999999998899
0.1261077281603452 0.0009999999999998899
0.09627370083257947 0.019083797152462573
0.20167633375965566 0.0009999999999998899
0.07863613343496578 0.11365144396785246
0.06879208293078648 0.25154114070599365
0.23260382242630329 0.0009999999999998899
0.2062311136189524 0.0009999999999998899
0.185443475

In [10]:


import scipy.stats as stats

query = "is_stroke == 1"
df_test = stroke.df.query(query)

for column in c:
    z, p = stats.kstest(stroke.df[column], "norm", args=(stroke.df[column].mean(), stroke.df[column].var()**0.5))
    print(z, p)


0.06368721203098826 5.471951487153302e-56
0.34480978024158443 0.0
0.02238185042134222 2.753693764858123e-07
0.04118717549348877 1.1768693697158923e-23
0.041970969743227005 1.5067773511499565e-24
0.3035149697996461 0.0
0.5330502032744238 0.0
0.42307347244454524 0.0
0.5029712348563808 0.0
0.5395097816780434 0.0
0.525884147413614 0.0
0.5272068573649002 0.0
0.5408131341400583 0.0
0.5409003535000171 0.0
0.5286881383921819 0.0
0.05210994265190938 1.291911210958456e-37
0.06166454755243711 1.635904889535339e-52
0.36936363499392855 0.0
0.14516193164286187 3.8921776023544324e-290
0.13801466153364628 3.4229252678235877e-262
0.20251393806635576 0.0
0.08839056576816873 1.5646771618884758e-107
0.08799923235326113 1.3875295328637483e-106
0.1391850552899513 1.1395420211744962e-266
0.19849652821185693 0.0
0.39123151831237507 0.0
0.4170312523751446 0.0
0.09050861134352067 9.794469878968742e-113
0.10625042242017435 2.6556862414711394e-155
0.09428244480545594 2.582936867506252e-122
0.2713190307029466 0.0


In [15]:
from scipy.stats import shapiro

query = "is_stroke == 0"
df_test = stroke.df.query(query)

shapiro(df_test)

ShapiroResult(statistic=0.24031978845596313, pvalue=0.0)

In [49]:
from scipy.stats import shapiro, normaltest, kstest, anderson

query = "is_stroke == 1"
df_test = stroke.df.query(query)

normal, not_normal = [], []
alpha = 0.05
for column in df_test.columns:
    stat, p = normaltest(df_test[column])  ## D'Agostino's K-squared test
    if p > alpha:
        normal.append(column)
    else:
        not_normal.append(column)

In [50]:
normal

['age',
 'height',
 'bmi',
 'preop_asa',
 'preop_hb_my',
 'preop_plt_sm',
 'preop_alb',
 'preop_k_sm',
 'opdur_cal_sm2',
 'andur_cal_my',
 'DBP_std']

In [77]:
query = "is_stroke == 1"
df_test = stroke.df.query(query)

alpha = 0.05
normal, not_normal = [], []
for column in df_test.columns:
    stat, p = kstest(df_test[column], "norm")  ## Kolmogorov-Smirnov test
    print("{:.6f}  {}".format(p, column))
    if p > alpha:
        normal.append(column)
    else:
        not_normal.append(column)

0.000000  age
0.000000  sex
0.000000  height
0.000000  weight
0.000000  bmi
0.000000  preop_asa
0.000000  preop_emop
0.000000  preop_htn
0.000000  preop_dm
0.000000  preop_cva
0.000000  preop_asthma
0.000000  preop_copd
0.000000  preop_liverds
0.000000  preop_kidneyds
0.000000  preop_tb
0.000000  preop_hb_my
0.000000  preop_plt_sm
0.000000  preop_cr_sm
0.000000  preop_na
0.000000  preop_alb
0.000000  preop_bun
0.000000  preop_k_sm
0.000000  preop_pt
0.000000  preop_ptt_sm
0.000000  preop_glucose_sm
0.000000  preop_gpt_sm
0.000000  preop_got
0.000000  preop_egfr
0.000000  opdur_cal_sm2
0.000000  andur_cal_my
0.000000  sum_uo
0.000000  sum_ebl
0.000000  sum_crystalloid
0.000000  sum_colloid
0.000000  sum_fluid
0.000000  sum_rbc_my
0.000000  sum_ffp
0.000000  sum_plt
0.000000  sum_cryo
0.000000  sum_tf
0.000000  sum_io
0.000000  SBP_below100
0.000000  SBP_below95
0.000000  SBP_below90
0.000000  SBP_below85
0.000000  SBP_below80
0.000000  SBP_below75
0.000000  SBP_below70
0.000000  SBP_bel

In [73]:
normal

[]

In [58]:
## Anderson-Darling test
query = "is_stroke == 1"
df_test = stroke.df.query(query)

alpha = 0.05
normal, not_normal = [], []
for column in df_test.columns:
    result = anderson(df_test[column], "norm")
    normality = 0
    for p in range(len(result.critical_values)):
        sl, cv = result.significance_level[p], result.critical_values[p]
        if result.statistic < result.critical_values[p]:
            normality += 1
        else:
            normality += 0
    if normality > 2.5:
        normal.append(column)
    else:
        not_normal.append(column)

In [59]:
normal

['height',
 'bmi',
 'preop_k_sm',
 'preop_pt',
 'opdur_cal_sm2',
 'andur_cal_my',
 'DBP_std']

In [84]:
query = "is_stroke == 0"
df_control = stroke.df.query(query)
query = "is_stroke == 1"
df_stroke = stroke.df.query(query)

import scipy.stats as stats

cat_columns = [
    "sex", "preop_asa", "preop_emop", "preop_htn", "preop_dm", "preop_cva",
    "preop_asthma", "preop_copd", "preop_liverds", "preop_kidneyds", "preop_tb"
]

for column in cat_columns:
    data = pd.crosstab(stroke.df[column], stroke.df.is_stroke)
    chi = stats.chi2_contingency(observed=data)
    print(column, chi[0], chi[1], chi[2])

sex 0.18986351155035125 0.6630302399338498 1
preop_asa 27.619439828821076 4.319423455017463e-05 5
preop_emop 14.827456883696861 0.0001178077841286605 1
preop_htn 2.860434657835813 0.09078296440720166 1
preop_dm 1.2681914636954468 0.2601062344336391 1
preop_cva 18.16064888691734 2.0303092525344945e-05 1
preop_asthma 0.0 1.0 1
preop_copd 0.5639879202449277 0.45265789664466893 1
preop_liverds 0.0 1.0 1
preop_kidneyds 0.00031783984737086314 0.9857760168974239 1
preop_tb 0.220165621781984 0.6389137518612673 1


In [87]:
df_control.median()

age            60.000000
sex             0.000000
height        162.100000
weight         62.400000
bmi            23.800000
                 ...    
HR_below35      0.000000
HR_mean        69.541885
HR_std          8.777779
opyear         18.000000
is_stroke       0.000000
Length: 100, dtype: float64

In [88]:
df_stroke.median()

age            61.000000
sex             0.000000
height        160.400000
weight         60.800000
bmi            23.400000
                 ...    
HR_below35      0.000000
HR_mean        70.375678
HR_std          8.398072
opyear         18.000000
is_stroke       1.000000
Length: 100, dtype: float64

In [90]:
from scipy.stats import mannwhitneyu

query = "is_stroke == 0"
control = stroke.df.query(query)

query = "is_stroke == 1"
target = stroke.df.query(query)

c_continuous = [
    "age", "weight", "height", "bmi",
    "preop_hb_my", "preop_plt_sm",
    "preop_cr_sm", "preop_na", "preop_alb", "preop_bun", "preop_k_sm",
    "preop_pt", "preop_ptt_sm", "preop_glucose_sm", "preop_gpt_sm", "preop_got",
    "preop_egfr",

    "opdur_cal_sm2", "andur_cal_my",
    "sum_uo", "sum_ebl", "sum_crystalloid", "sum_colloid", "sum_fluid", "sum_rbc_my",
    "sum_ffp", "sum_plt", "sum_cryo", "sum_tf", "sum_io",

    "SBP_below100", "SBP_below95", "SBP_below90", "SBP_below85", "SBP_below80",
    "SBP_below75", "SBP_below70", "SBP_below65", "SBP_below60", "SBP_below55",
    "SBP_below50", "SBP_mean", "SBP_std",
    "DBP_below70", "DBP_below65", "DBP_below60", "DBP_below55", "DBP_below50",
    "DBP_below45", "DBP_below40", "DBP_below35", "DBP_below30", "DBP_below25",
    "DBP_below20", "DBP_mean", "DBP_std",
    "SPO2_below95", "SPO2_below90", "SPO2_below85", "SPO2_below80", "SPO2_below75",
    "SPO2_below70", "SPO2_below65", "SPO2_below60", "SPO2_below55", "SPO2_below50",
    "SPO2_mean", "SPO2_std",
    "HR_above100", "HR_above105", "HR_above110", "HR_above115", "HR_above120",
    "HR_above125", "HR_above130", "HR_above135", "HR_above140", "HR_above145",
    "HR_above150", "HR_below60", "HR_below55", "HR_below50", "HR_below45",
    "HR_below40", "HR_below35", "HR_mean", "HR_std",
]

li = []
for column in c_continuous:
    mann = mannwhitneyu(control[column], target[column])

    li.append([column, mann[0], mann[1]])

pd.DataFrame(li, columns=["feature_name", "mann_statistics", "mann_p_value"])

,feature_name,mann_statistics,mann_p_value
0,age,831071.5,0.649874
1,weight,895480.5,0.364123
2,height,877367.5,0.599798
3,bmi,895861.0,0.359870
4,preop_hb_my,861413.0,0.851279
...,...,...,...
82,HR_below45,771692.5,0.039662
83,HR_below40,781698.0,0.024861
84,HR_below35,798472.5,0.037070
85,HR_mean,858076.0,0.906917


In [95]:
query = "is_stroke == 0"
df_control = stroke.df.query(query)
query = "is_stroke == 1"
df_target = stroke.df.query(query)

li = []
for column in cat_columns + c_continuous:
    control_mean, control_median, control_std = np.mean(df_control[column]), np.median(df_control[column]), np.std(df_control[column])
    target_mean, target_median, target_std = np.mean(df_target[column]), np.median(df_target[column]), np.std(df_target[column])

    li.append([column, control_median, control_mean, control_std, target_median, target_mean, target_std])

pd.DataFrame(li, columns=["feature_name", "control_median", "control_mean", "control_std", "target_median", "target_mean", "target_std"])

,feature_name,control_median,control_mean,control_std,target_median,target_mean,target_std
0,sex,0.000000,0.493448,0.499957,0.000000,0.467890,0.498968
1,preop_asa,2.000000,1.957489,0.660225,2.000000,2.100917,0.834409
2,preop_emop,0.000000,0.090584,0.287016,0.000000,0.201835,0.401370
3,preop_htn,0.000000,0.340344,0.473825,0.000000,0.422018,0.493881
4,preop_dm,0.000000,0.174071,0.379171,0.000000,0.128440,0.334579
...,...,...,...,...,...,...,...
93,HR_below45,0.000000,67.457904,330.050091,0.000000,178.532110,626.928680
94,HR_below40,0.000000,8.472160,64.028352,0.000000,11.449541,47.087560
95,HR_below35,0.000000,1.924439,14.619053,0.000000,1.761468,5.633939
96,HR_mean,69.541885,71.054365,12.171120,70.375678,71.638298,14.359318


In [96]:
class CoreUtils():
    def __init__(self):
        pass

    def check_label_ratio(self, df, column):
        neg, pos = np.bincount(df[column])
        total = neg + pos

        print("{}\n    Total: {}\n    Nagatives: {}\n    Positives: {} ({:.2f}% of dataframe)\n".format(
            column, total, neg, pos, (pos / total * 100)
        ))

    def compute_youden_index(self, model, X_test, y_test):
        y_pred = model.predict_proba(X_test)[:, 1]

        fpr, tpr, thresholds = roc_curve(y_test, y_pred)
        auroc = auc(fpr, tpr)

        J = tpr - fpr
        ix = np.argmax(J)
        best_threshold = thresholds[ix]

        y_pred = (y_pred >= best_threshold).astype(int)
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

        sen = tp / (tp + fn)
        spe = tn / (tn + fp)

        ppv = tp / (tp + fp)
        npv = tn / (tn + tn)
        print("ppv={:.3f} npv={:.3f}".format(ppv, npv))

        odd_ratio = (tp * tn) / (fp * fn)
        print("odd_ratio={:.3f}".format(odd_ratio))

        mean_fpr = np.linspace(0, 1, 100)

        tprs = []
        tprs.append(np.interp(mean_fpr, fpr, tpr))
        tprs[-1][0] = 0.0
        tprs = np.mean(tprs, axis=0)

        return tprs, fpr, tpr, auroc, best_threshold, ix, sen, spe

In [97]:
class BRMHStroke():
    def __init__(self, kfold=True, model_save=False, debug=False, shap=False):
        self.cores = CoreUtils()

        self.PATH_DATASET = None
        self.PATH_SAVE = None

        self.WITH_KFOLD = kfold
        self.WITH_SAVE = model_save
        self.WITH_MODEL_DEBUG = debug
        self.WITH_SHAP = shap

        self.c_pre, c_post = [], []
        self.c_hours = []

        self.df = None

        self.df_ttest_tp = None
        self.df_ttest_mean = None
        self.df_ttest_std = None

        self.df_train = None
        self.df_test = None
        self.X_train, self.y_train = None, None
        self.X_test, self.y_test = None, None

        self.rf_summary, self.rf_tprs, self.rf_summary_youden, self.rf_shap = [], [], [], []
        self.xgb_summary, self.xgb_tprs, self.xgb_summary_youden, self.xgb_shap = [], [], [], []
        self.lgb_summary, self.lgb_tprs, self.lgb_summary_youden, self.lgb_shap = [], [], [], []


    def set_path(self, path_dataset, path_save):
        self.PATH_DATASET = path_dataset
        self.PATH_SAVE = path_save


    def load_dataset(self):
        try:
            df = pd.read_csv(self.PATH_DATASET, sep=",")
        except:
            print("Error to open file")
            raise
        print(df.shape)

        df["sum_rbc_my"] = df["sum_rbc_my"].fillna(0)
        df["sum_tf"] = df["sum_ffp"] + df["sum_plt"] + df["sum_cryo"] + df["sum_rbc_my"]
        df["sum_io"] = df["sum_fluid"] - (df["sum_uo"] + df["sum_ebl"])

        self.df = df.copy()


    def drop_exclusion(self):
        df = self.df.copy()
        self.df = df.copy()


    def set_features(self):
        df = self.df.copy()

        self.c_basic = [
            "no", "ptno", "serialno", "opname"
        ]
        self.c_pre = [
            "age", "sex",
            "height", "weight", "bmi",
            "preop_asa", "preop_emop",
            "preop_htn", "preop_dm", "preop_cva", "preop_asthma", "preop_copd",
            "preop_liverds", "preop_kidneyds", "preop_tb", "preop_hb_my", "preop_plt_sm",
            "preop_cr_sm", "preop_na", "preop_alb", "preop_bun", "preop_k_sm",
            "preop_pt", "preop_ptt_sm", "preop_glucose_sm", "preop_gpt_sm", "preop_got",
            "preop_egfr"
        ]
        self.c_post = [
            "opdur_cal_sm", "andur_cal_my",
            "sum_uo", "sum_ebl", "sum_crystalloid", "sum_colloid", "sum_fluid", "sum_rbc_my",
            "sum_ffp", "sum_plt", "sum_cryo", "sum_tf", "sum_io",

            "SBP_below100", "SBP_below95", "SBP_below90", "SBP_below85", "SBP_below80",
            "SBP_below75", "SBP_below70", "SBP_below65", "SBP_below60", "SBP_below55",
            "SBP_below50", "SBP_mean", "SBP_std",

            "DBP_below70", "DBP_below65", "DBP_below60", "DBP_below55", "DBP_below50",
            "DBP_below45", "DBP_below40", "DBP_below35", "DBP_below30", "DBP_below25",
            "DBP_below20", "DBP_mean", "DBP_std",

            "SPO2_below95", "SPO2_below90", "SPO2_below85", "SPO2_below80", "SPO2_below75",
            "SPO2_below70", "SPO2_below65", "SPO2_below60", "SPO2_below55", "SPO2_below50",
            "SPO2_mean", "SPO2_std",

            "HR_above100", "HR_above105", "HR_above110", "HR_above115", "HR_above120",
            "HR_above125", "HR_above130", "HR_above135", "HR_above140", "HR_above145",
            "HR_above150", "HR_below60", "HR_below55", "HR_below50", "HR_below45",
            "HR_below40", "HR_below35", "HR_mean", "HR_std",
        ]

      
        df = df[self.c_basic + self.c_pre + self.c_post + ["is_stroke"]]

        self.df = df.copy()


    def fill_features(self, mode):
        df = self.df.copy()


        print("fill features original >> ", df.shape)
        self.cores.check_label_ratio(df, "is_stroke")


        df = df.drop_duplicates(subset=["ptno"], keep="first")
        print("dup >> ", df.shape)
        self.cores.check_label_ratio(df, "is_stroke")


        for column in df.columns:
            try:
                df[column] = df[column].astype(float)
            except:
                print(column)


        if mode == "pp":
            df = df[self.c_pre + self.c_post + ["is_stroke"]]
            print(df.shape)
        elif mode == "p":
            df = df[self.c_pre + ["is_stroke"]]
            print(df.shape)
        else:
            print("??")
            raise

        df = df.dropna()
        print("drop >> ", df.shape)

        self.df = df.copy()


    def compute_ttest(self):
        df = self.df.copy()

        query = "is_stroke == 0"
        df_control = df.query(query).drop(columns=["is_stroke"]).reset_index(drop=True)
        query = "is_stroke == 1"
        df_target = df.query(query).drop(columns=["is_stroke"]).reset_index(drop=True)

        li = []
        for column in df_control.columns:
            ret = stats.ttest_ind(df_control[column], df_target[column], equal_var=True)
            li.append([column, "{:.5f}".format(ret[0]), "{:.5f}".format(ret[1])])

        self.df_ttest_tp = pd.DataFrame(li, columns=["feature_name", "t_value", "p_value"])

        self.df_ttest_mean = pd.concat(
            [pd.DataFrame(np.mean(df_control), columns=["control"]),
             pd.DataFrame(np.mean(df_target), columns=["is_stroke"])],
            axis=1
        )

        self.df_ttest_std = pd.concat(
            [pd.DataFrame(np.std(df_control), columns=["control"]),
             pd.DataFrame(np.std(df_target), columns=["is_stroke"])],
            axis=1
        )


    def train_test_split(self):
        df = self.df.copy()

        self.X_test = df.drop(columns=["is_stroke"])
        self.y_test = df["is_stroke"]

        print(self.X_test.shape, self.y_test.shape)


    def plot_youden_index(self, rf_path, xgb_path, lgb_path, title):
        rf_model = joblib.load(rf_path)
        xgb_model = joblib.load(xgb_path)
        lgb_model = joblib.load(lgb_path)

        rf_tprs, rf_fpr, rf_tpr, rf_auroc, rf_thr, rf_ix, rf_sen, rf_spe = self.cores.compute_youden_index(rf_model, self.X_test, self.y_test)
        xgb_tprs, xgb_fpr, xgb_tpr, xgb_auroc, xgb_thr, xgb_ix, xgb_sen, xgb_spe = self.cores.compute_youden_index(xgb_model, self.X_test, self.y_test)
        lgb_tprs, lgb_fpr, lgb_tpr, lgb_auroc, lgb_thr, lgb_ix, lgb_sen, lgb_spe = self.cores.compute_youden_index(lgb_model, self.X_test, self.y_test)

        lw = 1

        FONT_SIZE = 12
        plt.rc("font", size=FONT_SIZE) # controls default text sizes
        plt.rc("axes", titlesize=FONT_SIZE) # fontsize of the axes title
        plt.rc("axes", labelsize=FONT_SIZE) # fontsize of the x and y labels
        plt.rc("xtick", labelsize=FONT_SIZE) # fontsize of the tick labels
        plt.rc("ytick", labelsize=FONT_SIZE) # fontsize of the tick labels
        plt.rc("legend", fontsize=FONT_SIZE) # legend fontsize
        plt.rc("figure", titlesize=FONT_SIZE) # fontsize of the figure title

        color_r, color_g, color_b = "crimson", "darkgreen", "darkblue"

        plt.figure(figsize=(9, 8))

        plt.plot(rf_fpr, rf_tpr, linestyle="-", lw=lw, alpha=1, color="k")
        plt.plot(xgb_fpr, xgb_tpr, linestyle="-", lw=lw, alpha=1, color="k")
        plt.plot(lgb_fpr, lgb_tpr, linestyle="-", lw=lw, alpha=1, color="k")

        plt.scatter(rf_fpr[rf_ix], rf_tpr[rf_ix], marker="+", s=300, lw=5, color=color_r)
        plt.scatter(xgb_fpr[xgb_ix], xgb_tpr[xgb_ix], marker="+", s=300, lw=5, color=color_g)
        plt.scatter(lgb_fpr[lgb_ix], lgb_tpr[lgb_ix], marker="+", s=300, lw=5, color=color_b)

        plt.plot([0, 1], [0, 1], linestyle="--", lw=lw, color="k")
        plt.xlim([-0.05, 1.05])
        plt.ylim([-0.05, 1.05])
        plt.xlabel("1-Specificity")
        plt.ylabel("Sensitivity")
        plt.title(title)

        red_patch = mpatches.Patch(
            color=color_r,
            label="RandomForest\nAUC={:.3f}  Th={:.3f}  Se={:.3f}  Sp={:.3f}".format(rf_auroc, rf_thr, rf_sen, rf_spe)
        )
        green_patch = mpatches.Patch(
            color=color_g,
            label="XGBoost\nAUC={:.3f}  Th={:.3f}  Se={:.3f}  Sp={:.3f}".format(xgb_auroc, xgb_thr, xgb_sen, xgb_spe)
        )
        blue_patch = mpatches.Patch(
            color=color_b,
            label="LightGBM\nAUC={:.3f}  Th={:.3f}  Se={:.3f}  Sp={:.3f}".format(lgb_auroc, lgb_thr, lgb_sen, lgb_spe)
        )

        plt.legend(loc="lower right", handles=[red_patch, green_patch, blue_patch])
        plt.show()


    # AUC comparison adapted from
    # https://github.com/Netflix/vmaf/
    def compute_midrank(self, x):
        """Computes midranks.
        Args:
           x - a 1D numpy array
        Returns:
           array of midranks
        """
        J = np.argsort(x)
        Z = x[J]
        N = len(x)
        T = np.zeros(N, dtype=np.float)
        i = 0
        while i < N:
            j = i
            while j < N and Z[j] == Z[i]:
                j += 1
            T[i:j] = 0.5*(i + j - 1)
            i = j
        T2 = np.empty(N, dtype=np.float)
        # Note(kazeevn) +1 is due to Python using 0-based indexing
        # instead of 1-based in the AUC formula in the paper
        T2[J] = T + 1
        return T2


    def fastDeLong(self, predictions_sorted_transposed, label_1_count):
        """
        The fast version of DeLong's method for computing the covariance of
        unadjusted AUC.
        Args:
           predictions_sorted_transposed: a 2D numpy.array[n_classifiers, n_examples]
              sorted such as the examples with label "1" are first
        Returns:
           (AUC value, DeLong covariance)
        Reference:
         @article{sun2014fast,
           title={Fast Implementation of DeLong's Algorithm for
                  Comparing the Areas Under Correlated Receiver Operating Characteristic Curves},
           author={Xu Sun and Weichao Xu},
           journal={IEEE Signal Processing Letters},
           volume={21},
           number={11},
           pages={1389--1393},
           year={2014},
           publisher={IEEE}
         }
        """
        # Short variables are named as they are in the paper
        m = label_1_count
        n = predictions_sorted_transposed.shape[1] - m
        positive_examples = predictions_sorted_transposed[:, :m]
        negative_examples = predictions_sorted_transposed[:, m:]
        k = predictions_sorted_transposed.shape[0]

        tx = np.empty([k, m], dtype=np.float)
        ty = np.empty([k, n], dtype=np.float)
        tz = np.empty([k, m + n], dtype=np.float)
        for r in range(k):
            tx[r, :] = self.compute_midrank(positive_examples[r, :])
            ty[r, :] = self.compute_midrank(negative_examples[r, :])
            tz[r, :] = self.compute_midrank(predictions_sorted_transposed[r, :])
        aucs = tz[:, :m].sum(axis=1) / m / n - float(m + 1.0) / 2.0 / n
        v01 = (tz[:, :m] - tx[:, :]) / n
        v10 = 1.0 - (tz[:, m:] - ty[:, :]) / m
        sx = np.cov(v01)
        sy = np.cov(v10)
        delongcov = sx / m + sy / n
        return aucs, delongcov


    def calc_pvalue(self, aucs, sigma):
        """Computes log(10) of p-values.
        Args:
           aucs: 1D array of AUCs
           sigma: AUC DeLong covariances
        Returns:
           log10(pvalue)
        """
        l = np.array([[1, -1]])
        z = np.abs(np.diff(aucs)) / np.sqrt(np.dot(np.dot(l, sigma), l.T))
        #    p = np.log10(2) + scipy.stats.norm.logsf(z, loc=0, scale=1) / np.log(10)
        p = 2 * (1 - scipy.stats.norm.cdf(z, loc=0, scale=1))
        return z, p

    def compute_ground_truth_statistics(self, ground_truth):
        assert np.array_equal(np.unique(ground_truth), [0, 1])
        order = (-ground_truth).argsort()
        label_1_count = int(ground_truth.sum())
        return order, label_1_count


    def delong_roc_variance(self, ground_truth, predictions):
        """
        Computes ROC AUC variance for a single set of predictions
        Args:
           ground_truth: np.array of 0 and 1
           predictions: np.array of floats of the probability of being class 1
        """
        order, label_1_count = self.compute_ground_truth_statistics(ground_truth)
        predictions_sorted_transposed = predictions[np.newaxis, order]
        aucs, delongcov = self.fastDeLong(predictions_sorted_transposed, label_1_count)
        assert len(aucs) == 1, "There is a bug in the code, please forward this to the developers"
        return aucs[0], delongcov


    def delong_roc_test(self, ground_truth, predictions_one, predictions_two):
        """
        Computes log(p-value) for hypothesis that two ROC AUCs are different
        Args:
           ground_truth: np.array of 0 and 1
           predictions_one: predictions of the first model,
              np.array of floats of the probability of being class 1
           predictions_two: predictions of the second model,
              np.array of floats of the probability of being class 1
        """
        order, label_1_count = self.compute_ground_truth_statistics(ground_truth)
        predictions_sorted_transposed = np.vstack((predictions_one, predictions_two))[:, order]
        aucs, delongcov = self.fastDeLong(predictions_sorted_transposed, label_1_count)
        return self.calc_pvalue(aucs, delongcov)

    def print_delong_performance(self, p_path, pp_path, X_test_p, y_test_p, X_test_pp, y_test_pp):
        p_best_model = joblib.load(p_path)
        pp_best_model = joblib.load(pp_path)

        p_pred = p_best_model.predict_proba(X_test_p)[:, 1]
        pp_pred = pp_best_model.predict_proba(X_test_pp)[:, 1]

        p_pp_z, p_pp_p = self.delong_roc_test(y_test_p, p_pred, pp_pred)
        print(p_pp_z, p_pp_p)

        self.p_pp_delong_z = p_pp_z[0][0]
        self.p_pp_delong_p = p_pp_p[0][0]


In [99]:
path_dataset = os.path.join(os.getcwd(), "data", "220420_BRMH_vitaldb_n568_inclusion_stroke_filled.csv")

stroke_pp = BRMHStroke(kfold=True, model_save=True, debug=True, shap=True)
stroke_pp.set_path(path_dataset, None)
stroke_pp.load_dataset()

stroke_pp.drop_exclusion()
stroke_pp.set_features()
stroke_pp.fill_features(mode="pp")
stroke_pp.compute_ttest()
stroke_pp.train_test_split()

(568, 103)
fill features original >>  (568, 103)
is_stroke
    Total: 568
    Nagatives: 554
    Positives: 14 (2.46% of dataframe)

dup >>  (506, 103)
is_stroke
    Total: 506
    Nagatives: 493
    Positives: 13 (2.57% of dataframe)

serialno
opname
(506, 99)
drop >>  (460, 99)
(460, 98) (460,)


In [104]:
query = "is_stroke == 1"
df_test = stroke_pp.df.query(query)

alpha = 0.05
normal, not_normal = [], []
for column in df_test.columns:
    stat, p = kstest(df_test[column], "norm")  ## Kolmogorov-Smirnov test
    print("{:.6f}  {}".format(p, column))
    if p > alpha:
        normal.append(column)
    else:
        not_normal.append(column)

0.000000  age
0.001574  sex
0.000000  height
0.000000  weight
0.000000  bmi
0.000000  preop_asa
0.001574  preop_emop
0.001574  preop_htn
0.001574  preop_dm
0.000035  preop_cva
0.001574  preop_asthma
0.001574  preop_copd
0.001574  preop_liverds
0.001574  preop_kidneyds
0.001574  preop_tb
0.000000  preop_hb_my
0.000000  preop_plt_sm
0.000010  preop_cr_sm
0.000000  preop_na
0.000000  preop_alb
0.000000  preop_bun
0.000000  preop_k_sm
0.000000  preop_pt
0.000000  preop_ptt_sm
0.000000  preop_glucose_sm
0.000000  preop_gpt_sm
0.000000  preop_got
0.000000  preop_egfr
0.000000  opdur_cal_sm
0.000000  andur_cal_my
0.000000  sum_uo
0.000000  sum_ebl
0.000000  sum_crystalloid
0.001574  sum_colloid
0.000000  sum_fluid
0.001574  sum_rbc_my
0.001574  sum_ffp
0.001574  sum_plt
0.001574  sum_cryo
0.001574  sum_tf
0.000000  sum_io
0.000000  SBP_below100
0.000000  SBP_below95
0.000000  SBP_below90
0.000000  SBP_below85
0.000000  SBP_below80
0.000000  SBP_below75
0.000000  SBP_below70
0.000000  SBP_belo

In [103]:
normal

[]

In [106]:
query = "is_stroke == 0"
df_control = stroke_pp.df.query(query)
query = "is_stroke == 1"
df_stroke = stroke_pp.df.query(query)

import scipy.stats as stats

cat_columns = [
    "sex", "preop_asa", "preop_emop", "preop_htn", "preop_dm", "preop_cva",
    "preop_asthma", "preop_copd", "preop_liverds", "preop_kidneyds", "preop_tb"
]
li = []
for column in cat_columns:
    data = pd.crosstab(stroke_pp.df[column], stroke_pp.df.is_stroke)
    chi = stats.chi2_contingency(observed=data)

    li.append([column, chi[0], chi[1], chi[2]])

pd.DataFrame(li, columns=["feature_name", "X2", "p-value", "freedom"])

,feature_name,X2,p-value,freedom
0,sex,0.001012,9.746264e-01,1
1,preop_asa,5.423208,1.433044e-01,3
2,preop_emop,46.352803,9.876613e-12,1
3,preop_htn,2.887565,8.926573e-02,1
4,preop_dm,0.058005,8.096772e-01,1
5,preop_cva,19.474310,1.019618e-05,1
6,preop_asthma,0.000000,1.000000e+00,1
7,preop_copd,0.000000,1.000000e+00,1
8,preop_liverds,1.107490,2.926281e-01,1
9,preop_kidneyds,0.000000,1.000000e+00,1


In [108]:
from scipy.stats import mannwhitneyu

query = "is_stroke == 0"
control = stroke_pp.df.query(query)
query = "is_stroke == 1"
target = stroke_pp.df.query(query)

c_continuous = [
    "age", "weight", "height", "bmi",
    "preop_hb_my", "preop_plt_sm",
    "preop_cr_sm", "preop_na", "preop_alb", "preop_bun", "preop_k_sm",
    "preop_pt", "preop_ptt_sm", "preop_glucose_sm", "preop_gpt_sm", "preop_got",
    "preop_egfr",

    "opdur_cal_sm", "andur_cal_my",
    "sum_uo", "sum_ebl", "sum_crystalloid", "sum_colloid", "sum_fluid", "sum_rbc_my",
    "sum_ffp", "sum_plt", "sum_cryo", "sum_tf", "sum_io",

    "SBP_below100", "SBP_below95", "SBP_below90", "SBP_below85", "SBP_below80",
    "SBP_below75", "SBP_below70", "SBP_below65", "SBP_below60", "SBP_below55",
    "SBP_below50", "SBP_mean", "SBP_std",
    "DBP_below70", "DBP_below65", "DBP_below60", "DBP_below55", "DBP_below50",
    "DBP_below45", "DBP_below40", "DBP_below35", "DBP_below30", "DBP_below25",
    "DBP_below20", "DBP_mean", "DBP_std",
    "SPO2_below95", "SPO2_below90", "SPO2_below85", "SPO2_below80", "SPO2_below75",
    "SPO2_below70", "SPO2_below65", "SPO2_below60", "SPO2_below55", "SPO2_below50",
    "SPO2_mean", "SPO2_std",
    "HR_above100", "HR_above105", "HR_above110", "HR_above115", "HR_above120",
    "HR_above125", "HR_above130", "HR_above135", "HR_above140", "HR_above145",
    "HR_above150", "HR_below60", "HR_below55", "HR_below50", "HR_below45",
    "HR_below40", "HR_below35", "HR_mean", "HR_std",
]

li = []
for column in c_continuous:
    mann = mannwhitneyu(control[column], target[column])
    li.append([column, mann[0], mann[1]])

pd.DataFrame(li, columns=["feature_name", "mann_statistics", "mann_p_value"])

,feature_name,mann_statistics,mann_p_value
0,age,2803.0,0.829079
1,weight,3317.0,0.384364
2,height,2437.5,0.322423
3,bmi,3551.5,0.171879
4,preop_hb_my,2976.5,0.881365
...,...,...,...
82,HR_below45,3887.0,0.037867
83,HR_below40,3922.0,0.031526
84,HR_below35,3970.0,0.024325
85,HR_mean,1550.0,0.004133


In [109]:
query = "is_stroke == 0"
df_control = stroke_pp.df.query(query)
query = "is_stroke == 1"
df_target = stroke_pp.df.query(query)

li = []
for column in cat_columns + c_continuous:
    control_mean, control_median, control_std = np.mean(df_control[column]), np.median(df_control[column]), np.std(df_control[column])
    target_mean, target_median, target_std = np.mean(df_target[column]), np.median(df_target[column]), np.std(df_target[column])

    li.append([column, control_median, control_mean, control_std, target_median, target_mean, target_std])

pd.DataFrame(li, columns=["feature_name", "control_median", "control_mean", "control_std", "target_median", "target_mean", "target_std"])

,feature_name,control_median,control_mean,control_std,target_median,target_mean,target_std
0,sex,1.000000,0.505593,0.499969,0.000000,0.461538,0.498519
1,preop_asa,2.000000,2.123043,0.539767,2.000000,2.384615,0.624926
2,preop_emop,0.000000,0.031320,0.174181,0.000000,0.461538,0.498519
3,preop_htn,0.000000,0.429530,0.495009,0.000000,0.153846,0.360801
4,preop_dm,0.000000,0.221477,0.415240,0.000000,0.153846,0.360801
...,...,...,...,...,...,...,...
93,HR_below45,203.000000,255.411633,212.172855,125.000000,192.307692,249.088302
94,HR_below40,191.000000,232.888143,180.712141,123.000000,182.923077,247.621753
95,HR_below35,187.000000,225.317673,174.260964,121.000000,176.692308,245.391488
96,HR_mean,64.488527,65.307408,13.496273,77.831760,78.395060,14.633059


In [172]:
X_columns = stroke_pp.X_train.columns

li = []
for column in X_columns:
    ret = stats.ttest_ind(stroke_pp.X_train[column], stroke_pp.X_test[column], equal_var=True)
    li.append([column, "{:.5f}".format(ret[0]), "{:.5f}".format(ret[1])])

ttest_tp = pd.DataFrame(li, columns=["feature_name", "t_value", "p_value"])

ttest_mean = pd.concat(
    [pd.DataFrame(np.mean(stroke_pp.X_train), columns=["SNUH_train"]),
     pd.DataFrame(np.mean(stroke_pp.X_test), columns=["SNUH_test"])],
    axis=1
)

ttest_std = pd.concat(
    [pd.DataFrame(np.std(stroke_pp.X_train), columns=["SNUH_train"]),
     pd.DataFrame(np.std(stroke_pp.X_test), columns=["SNUH_test"])],
    axis=1
)

In [173]:
ttest_tp

,feature_name,t_value,p_value
0,age,-0.59951,0.54884
1,sex,4.12183,0.00004
2,height,2.45455,0.01412
3,weight,-1.72837,0.08394
4,bmi,-3.84070,0.00012
...,...,...,...
93,HR_below45,-2.91265,0.00359
94,HR_below40,-3.44129,0.00058
95,HR_below35,-5.91449,0.00000
96,HR_mean,3.30670,0.00095


In [175]:
ttest_std

,SNUH_train,SNUH_test
age,14.790162,14.708557
sex,0.499990,0.498845
height,8.671180,8.657025
weight,11.857902,12.139947
bmi,3.636978,3.762922
...,...,...
HR_below45,309.371251,390.942893
HR_below40,53.979734,85.476804
HR_below35,11.531726,20.727868
HR_mean,12.272146,11.935803


In [201]:
female, male = stroke_pp.X_train["sex"].value_counts()[0], stroke_pp.X_train["sex"].value_counts()[1]
print("{} {:.2f}   {} {:.2f}".format(female, female / stroke_pp.X_train.shape[0] * 100, male, male / stroke_pp.X_train.shape[0] * 100))
print(stroke_pp.X_train.shape)

5742 49.68   5815 50.32
(11557, 98)


In [190]:
female, male = stroke_pp.X_test["sex"].value_counts()[0], stroke_pp.X_test["sex"].value_counts()[1]
print("{} {:.2f}   {} {:.2f}".format(female, female / stroke_pp.X_test.shape[0] * 100, male, male / stroke_pp.X_test.shape[0] * 100))
print(stroke_pp.X_test.shape)

2240 53.40   1955 46.60
(4195, 98)


In [187]:
column = "preop_tb"
print(column)

res = stroke_pp.X_train[column].value_counts().reset_index()
res["percent"] = res[column] / stroke_pp.X_train.shape[0] * 100
res.sort_values(by="index")

preop_tb


,index,preop_tb,percent
0,0.0,11450,99.074154
1,1.0,107,0.925846


In [200]:
column = "preop_tb"
print(column)

res = stroke_pp.X_test[column].value_counts().reset_index()
res["percent"] = res[column] / stroke_pp.X_test.shape[0] * 100
res.sort_values(by="index")

preop_tb


,index,preop_tb,percent
0,0.0,4164,99.261025
1,1.0,31,0.738975


In [239]:
X_columns = stroke_pp.X_train.columns

li = []
for column in X_columns:
    ret = stats.ttest_ind(stroke_pp.df[column], stroke.df[column], equal_var=False)
    li.append([column, "{:.5f}".format(ret[0]), "{:.5f}".format(ret[1])])

ttest_tp = pd.DataFrame(li, columns=["feature_name", "t_value", "p_value"])

ttest_mean = pd.concat(
    [pd.DataFrame(np.mean(stroke_pp.df), columns=["SNUH"]),
     pd.DataFrame(np.mean(stroke.df), columns=["BMC"])],
    axis=1
)

ttest_std = pd.concat(
    [pd.DataFrame(np.std(stroke_pp.df), columns=["SNUH"]),
     pd.DataFrame(np.std(stroke.df), columns=["BMC"])],
    axis=1
)

In [240]:
ttest_tp

,feature_name,t_value,p_value
0,age,-5.92207,0.00000
1,sex,-1.30620,0.19273
2,height,2.60084,0.00988
3,weight,2.76965,0.00605
4,bmi,1.17280,0.24204
...,...,...,...
93,HR_below45,-13.13002,0.00000
94,HR_below40,-17.90301,0.00000
95,HR_below35,-18.10043,0.00000
96,HR_mean,3.33355,0.00099


In [216]:
female, male = stroke_pp.df["sex"].value_counts()[0], stroke_pp.df["sex"].value_counts()[1]
print("{} {:.2f}   {} {:.2f}".format(female, female / stroke_pp.df.shape[0] * 100, male, male / stroke_pp.df.shape[0] * 100))
print(stroke_pp.df.shape)

female, male = stroke.df["sex"].value_counts()[0], stroke.df["sex"].value_counts()[1]
print("{} {:.2f}   {} {:.2f}".format(female, female / stroke.df.shape[0] * 100, male, male / stroke.df.shape[0] * 100))
print(stroke.df.shape)

7982 50.67   7770 49.33
(15752, 100)
109 46.38   126 53.62
(235, 99)


In [228]:
column = "preop_tb"
print(column)

res = stroke_pp.df[column].value_counts().reset_index()
res["percent"] = res[column] / stroke_pp.df.shape[0] * 100
res.sort_values(by="index")

preop_tb


,index,preop_tb,percent
0,0.0,15614,99.123921
1,1.0,138,0.876079


In [238]:
column = "preop_tb"
print(column)

res = stroke.df[column].value_counts().reset_index()
res["percent"] = res[column] / stroke.df.shape[0] * 100
res.sort_values(by="index")

preop_tb


,index,preop_tb,percent
0,0.0,217,92.340426
1,1.0,18,7.659574


In [88]:
path_n568 = os.path.join(os.getcwd(), "data", "220530_BRMH_vitaldb_n568_inclusion_stroke_filled_cut.csv")
path_brmh = os.path.join(os.getcwd(), "data", "20220217_AN김태경교수님.csv")

df_brmh = pd.read_csv(path_brmh).iloc[1:]

c = {"등록번호":"ptno", "Unnamed: 55":"preop_pt"}
df_brmh = df_brmh.rename(columns=c)[["ptno", "serialno", "preop_pt"]]
df_brmh["ptno"] = df_brmh["ptno"].astype(float)
df_brmh.head()

,ptno,serialno,preop_pt
1,1613508.0,2020-10-01_ 2_13.000,NaN
2,1652612.0,2020-10-01_ 2_14.033,96.7
3,757955.0,2020-10-01_ 2_23.038,NaN
4,1613508.0,2020-10-01_ 2_17.015,NaN
5,1153255.0,2020-10-01_ 7_8.022,90.5


In [91]:
a = stroke.df[["ptno", "serialno"]]
b = df_brmh

c = pd.merge(a, b, how="left", on=["ptno", "serialno"])
c["preop_pt"] = c["preop_pt"].astype(float)
mean = c["preop_pt"].mean()
std = c["preop_pt"].std()
print("{:.2f} {:.2f}".format(mean, std))

91.09 12.37


In [85]:
b

,ptno,serialno,serialno,preop_pt
1,1613508.0,2020-10-01_ 2_13.000,2020-10-01_ 2_13.000,NaN
2,1652612.0,2020-10-01_ 2_14.033,2020-10-01_ 2_14.033,96.7
3,757955.0,2020-10-01_ 2_23.038,2020-10-01_ 2_23.038,NaN
4,1613508.0,2020-10-01_ 2_17.015,2020-10-01_ 2_17.015,NaN
5,1153255.0,2020-10-01_ 7_8.022,2020-10-01_ 7_8.022,90.5
...,...,...,...,...
2850,451115.0,2021-01-13_13_10.048,2021-01-13_13_10.048,90.6
2851,1699425.0,2021-01-13_13_13.050,2021-01-13_13_13.050,96.5
2852,1714586.0,2021-01-13_14_7.050,2021-01-13_14_7.050,82.6
2853,1282497.0,2021-01-13_15_15.045,2021-01-13_15_15.045,81.6
